# 環境構築（最初に実行）


参考：\
https://github.com/googlecolab/colabtools/issues/3347 \
https://www.teijitaisya.com/python-gspread/

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium, requests
pip install selenium
pip install requests
pip install selenium-wire
pip install beautifulsoup4

# Install gspread
pip install --upgrade gspread gspread-dataframe
pip install oauth2client

# Install tenacity, timeout-decorator, func_timeout
pip install tenacity
pip install timeout-decorator
pip install func_timeout

Executing: /tmp/apt-key-gpghome.gaHLM9ceXx/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.ifRxFa9g0e/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.EEIqHc0c7W/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [2]:
from seleniumwire import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from gspread.exceptions import APIError
#from gspread_dataframe import set_with_dataframe
import os, re
import time
import pandas as pd
from datetime import datetime, date
#from zoneinfo import ZoneInfo
import numpy as np
import itertools
from tenacity import retry, retry_if_exception_type, stop_after_attempt, wait_fixed, before_sleep, RetryError#, stop_after_delay
from func_timeout import func_timeout, FunctionTimedOut
#from timeout_decorator import timeout, TimeoutError
import requests
#from http.cookiejar import CookieJar
#import random
#import string
import json
#from bs4 import BeautifulSoup
from openpyxl.utils import get_column_letter
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import threading
import sys
import traceback
from tqdm import tqdm

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# スクレイピング関数（次に実行）

## 必要関数・クラス

### 情報記入シート作成


参考：\
https://note.nkmk.me/python-datetime-now-today/ \
https://zenn.dev/fujimotoshinji/scraps/f9c25aeb00a716 \
https://www.teijitaisya.com/python-gspread/

In [21]:
class MakeSpreadSheet():
    def __init__(self, area, disease, key = None, sheetname1 = None, sheetname2 = None):
        self.area = area
        self.disease = disease
        self.key = key
        self.sheetname1 = sheetname1
        self.sheetname2 = sheetname2

    def make_spreadsheet(self):
        if self.key ==  None:
          dt_now = datetime.now(ZoneInfo(self.area)).strftime("%Y-%m-%dT%H:%M")
          filename = self.disease + '_' + dt_now
          ss = gc.create(filename)
        else:
          ss = gc.open_by_key(self.key)

        print(ss.title)

        wb = gc.open(str(ss.title))
        if (self.sheetname1 == None) or (self.sheetname2 == None):
            if self.sheetname1 == None:
              wb.add_worksheet(title="Summary", rows=1000, cols=500)
              ws1 = wb.worksheet('Summary')
            if self.sheetname2 == None:
              wb.add_worksheet(title="Detail", rows=1000, cols=500)
              ws2 = wb.worksheet('Detail')
        else:
          ws1 = wb.worksheet(self.sheetname1)
          ws2 = wb.worksheet(self.sheetname2)

        return ss, ws1, ws2

### 日付比較

In [22]:
class CompareDate():
    def narrow_down(self, date_list):
        if (None in date_list) or ('-' in date_list):
            return False

        date_1 = pd.to_datetime(date_list[0]).date() if isinstance(date_list[0], str) else date_list[0]
        date_2 = pd.to_datetime(date_list[1]).date() if isinstance(date_list[1], str) else date_list[1]
        if date_1 < date_2:
            return True
        else: return False

### cURLリクエスト送信

In [23]:
class SendRequest():
  # cURLリクエストを送信する関数
    def send_request_with_cookie(self, query_param, command, ID = None):
        url = query_param["url"]
        if ID:
          matches = re.findall(r'/(\d+)/', url)
          for i, match in enumerate(matches):
              if i < len(ID):
                  new_event_id = ID[i]
                  url = re.sub(rf'/{match}/', f'/{new_event_id}/', url, count=1)
        #print(url)

        headers = {
            'accept': 'application/json',
            'content-type': 'application/json'
        }
        '''
        cookies = {}
        for cookie in query_param["headers"]["cookie"].split('; '):
            name, value = cookie.split('=')
            cookies[name] = value
        '''
        if query_param["body"]:
            data = query_param["body"]
        else:
            data = {}
        if command == 'POST':
            #response = requests.post(url, headers=headers, json=data, cookies=cookies)
            response = requests.post(url, headers=headers, json=data)
        elif command == 'GET':
            #response = requests.get(url, headers=headers, json=data, cookies=cookies)
            response = requests.get(url, headers=headers, json=data)
        return response

### ブラウザ使用（API変更時に起動）



参考（フィルター適用）:\
https://stackoverflow.com/questions/74973982/selenium-how-to-find-an-element-by-part-of-the-placeholder-value \
https://prcmyself.com/selenium-for-text-input-and-deletion-of-input-forms \
https://www.seleniumqref.com/api/python/element_set/Python_special_send_keys.html \
https://teratail.com/questions/224497 \
https://qiita.com/ShortArrow/items/ced7bf23b806c0d835da \
https://ja.stackoverflow.com/questions/51940/python-%E4%B8%80%E5%AE%9A%E6%99%82%E9%96%93%E3%81%A7%E5%87%A6%E7%90%86%E3%81%8C%E7%B5%82%E4%BA%86%E3%81%97%E3%81%AA%E3%81%91%E3%82%8C%E3%81%B0-%E6%AC%A1%E3%81%AE%E5%87%A6%E7%90%86%E3%81%AB%E7%A7%BB%E3%82%8B%E3%82%88%E3%81%86%E3%81%AA%E3%82%B3%E3%83%BC%E3%83%89 \
https://zenn.dev/taroman_zenn/articles/dd0b33a3a37d1e

参考（payload）:\
https://medium.com/@upendraprasadmahto652/selenium-wire-intercept-and-analyze-network-information-in-automation-testing-ffe3b4d64215

参考（ソート）:\
https://qiita.com/n10432/items/e0315979286ea9121d57

In [24]:
class GetRequestParam():
    def __init__(self):
        self.service = Service("/usr/bin/chromedriver")
        self.options = Options()                                        # ブラウザをheadlessモード実行
        self.options.add_argument("--headless")                        # ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
        self.options.add_argument("--no-sandbox")                     # サンドボックスモードの解除。これも必須。
        self.options.add_argument('--disable-dev-shm-usage')           # これも設定した方がよい。
        self.options.add_argument('--disk-cache-dir=/path/to/cache')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.driver = self.initialize_driver()

    def before_retry(retry_state):
        print(f"Retrying to Connect... Attempt number: {retry_state.attempt_number}")

    @retry(retry=retry_if_exception_type(TimeoutError), stop=stop_after_attempt(3), wait=wait_fixed(2), before_sleep=before_retry)
    def initialize_driver(self):
        try:
            driver = func_timeout(15, self._initialize_driver)
        except FunctionTimedOut:
            raise TimeoutError("The initialize_driver method timed out")
        driver.set_window_size(2000, 1500)
        return driver

    def _initialize_driver(self):
        self.reset_environment()

        try:
            driver = webdriver.Chrome(service=self.service, options=self.options)
            return driver
        except WebDriverException as e:
            if "ERR_CONNECTION_CLOSED" in str(e):
                print("Connection closed error occurred. Retrying...")
                raise TimeoutError("Connection closed during initialization")
            else:
                raise e

    def reset_environment(self):
        os.system("pkill -f chromedriver")
        os.system("pkill -f chrome")

    def scraping_param(self):
        # キャプチャされたリクエストを取得
        requests_data = []
        self.capture_request(requests_data, 'filtered-list')
        filtering_query = requests_data[-1]

        self.go_to_preview()

        self.capture_request(requests_data, 'all-information')
        preview_query = requests_data[-1]

        self.go_to_detail()

        self.capture_request(requests_data, 'all-information')
        detail_query = requests_data[-1]

        return filtering_query, preview_query, detail_query

    def capture_request(self, requests_data, target):
        for request in self.driver.requests:
            if request.response:
                # 特定のAPIエンドポイントに送信されたリクエストをフィルタリング
                if target in request.url:
                    request_data = {
                        'url': request.url,
                        'method': request.method,
                        'headers': request.headers,
                        'body': json.loads(request.body.decode('utf-8')) if request.body else None
                    }
                    requests_data.append(request_data)

    def go_to_preview(self):
        uptodate_report = self.driver.find_elements(By.XPATH, "//button[contains(., 'more_horiz')]")
        self.driver.execute_script("arguments[0].click();", uptodate_report[0])
        #uptodate_report.click()
        WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Preview last report')]"))
        )
        #time.sleep(3)

        preview = self.driver.find_element(By.XPATH, "//button[contains(., 'Preview last report')]")
        preview.click()
        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'OUTBREAKS')]"))
        )
        #time.sleep(5)

    def go_to_detail(self):
        report_row_list = self.driver.find_elements(By.XPATH, "//span[contains(text(), 'OUTBREAKS')]")
        target_column = len(report_row_list) - 1
        if target_column < 0:
          print("OUTBREAKSのスパン要素が見つかりません。")
          return

        report_row = report_row_list[target_column].find_element(By.XPATH, "../../..")
        report_row.click()
        WebDriverWait(report_row, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "mat-expansion-panel-header"))
        )

        reports = report_row.find_elements(By.TAG_NAME, "mat-expansion-panel-header")
        number2 = len(reports)
        if number2 == 0:
          print("レポート情報が見つかりません。")
          return

        target_report = number2 - 1
        attention = reports[target_report]
        attention.click()
        time.sleep(5)

In [25]:
class HeadlessBrowser(GetRequestParam):
    # main page
    def __init__(self):
        self.url = "https://wahis.woah.org/#/event-management"
        super().__init__()

    def before_retry_action(retry_state):
        print(f"Retrying to visit url... Attempt number: {retry_state.attempt_number}")

    @retry(retry=retry_if_exception_type(TimeoutError), stop=stop_after_attempt(3), wait=wait_fixed(2), before_sleep=before_retry_action)
    def visit_url(self, filter):
        try:
            result = func_timeout(50, self._visit_url, args=(filter,))
        except FunctionTimedOut:
            raise TimeoutError("The visit_url method timed out")
        return result

    def _visit_url(self, filter):
        self.driver.get(self.url)
        time.sleep(5)
        try:
          self.close_cookie_pref()
        except NoSuchElementException as e:
          pass
        time.sleep(5)
        filtering_url = []
        for item in ['country', 'disease', 'catalog', 'reports-count']:
                self.capture_request(filtering_url, item)
        ref_urls = [sublist['url'] for sublist in filtering_url]
        target_list = ["country/list?","disease/first-level-filters", "disease/second-level-filters", "catalog/report-reason",
                         "catalog/event-status", "catalog/report-status", "country/list-geo-region", "event/reports-count"]
        if not all(any(target in urls for urls in ref_urls) for target in target_list):
            self.scroll_to_top()
            time.sleep(1)
            self.apply_filters()
            self.select_filter(filter)
            self.close_dialog()
            time.sleep(1)
            ref_urls = []
        else:
            ref_urls.sort(key=lambda url: next((i for i, target in enumerate(target_list) if target in url), len(target_list)))

        FQ, PQ, DQ = self.scraping_param()
        print(FQ)
        print(PQ)
        print(DQ)
        print(ref_urls)

        # WebDriverを終了
        self.driver.quit()

        return FQ, PQ, DQ, ref_urls

    # delete cookie popup
    def close_cookie_pref(self):
        cookie = self.driver.find_element(By.XPATH, "//div[contains(@class, 'close-cookie-pref')]")
        self.driver.execute_script("arguments[0].click();", cookie)
        time.sleep(3)

    def scroll_to_top(self):
        self.driver.execute_script("window.scrollTo(0, 0)")

    def apply_filters(self):
        try:
            filter_button = WebDriverWait(self.driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Filters')]"))
            )
            self.driver.execute_script("arguments[0].click();", filter_button)
        except TimeoutException:
            print("The filter button was not found or clickable.")
        #filter_button = self.driver.find_element(By.XPATH, "//button[contains(., 'Filters')]")
        #self.driver.execute_script("arguments[0].click();", filter_button)

    def select_filter(self, filter):
        for key, value in filter.items():
            self.select_argu(key, value)

    def select_argu(self, filter_name, filter_value):
        if filter_value != None:
            filter_field = self.driver.find_element(By.XPATH, f"//div[mat-form-field[contains(., '{filter_name}')]]")
            filter_field.click()
            time.sleep(3)
            for item in filter_value:
                TARGET = self.driver.find_element(By.XPATH, "//mat-option[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{}')]".format(item.lower()))
                TARGET.click()
                time.sleep(3)
            return
        else:
            return

    def close_dialog(self):
        close_button = self.driver.find_element(By.XPATH, "//button[contains(., 'close')]")
        self.driver.execute_script("arguments[0].click();", close_button)

    def close(self):
        self.driver.quit()

### API保管用

In [26]:
class StoreAPI():
    contry_url = 'https://wahis.woah.org/api/v1/pi/country/list?language=en' # 国ID
    disease_url = 'https://wahis.woah.org/api/v1/pi/disease/first-level-filters?language=en' # 疾病ID
    subtype_url = 'https://wahis.woah.org/api/v1/pi/disease/second-level-filters?language=en' # サブタイプID
    reason_url = 'https://wahis.woah.org/api/v1/pi/catalog/report-reason/list?language=en' # reason ID
    status_url = 'https://wahis.woah.org/api/v1/pi/catalog/event-status/list?language=en' # event status ID
    draft_url = 'https://wahis.woah.org/api/v1/pi/catalog/report-status/list?language=en' # Draft ID
    region_url = 'https://wahis.woah.org/api/v1/pi/country/list-geo-region?language=en' # region ID
    reportnum_url = 'https://wahis.woah.org/api/v1/pi/event/reports-count' # Report ID
    ref_urls = [contry_url, disease_url, subtype_url, reason_url, status_url, draft_url, region_url, reportnum_url]

    filtered_url = 'https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en' # Filtered list
    preview_url = 'https://wahis.woah.org/api/v1/pi/review/event/0000/all-information?language=en' # preview list
    report_url = 'https://wahis.woah.org/api/v1/pi/review/event/0000/outbreak/111111/all-information?language=en' # report

    def __init__(self, filter):
        self.filter = filter

    def check_API(self):
        response = requests.get(StoreAPI.contry_url) # 代表としてAPIの有用性の確認
        if response.status_code != 200:
            print("API url is changed. You should rewrite this class variables.")
            n = 0
            while True:
              try:
                  if n == 0:
                      browser = HeadlessBrowser()
                      FQ_param, PQ_param, DQ_param, ref_urls = browser.visit_url(self.filter)
                      break
                  else:
                      print("Restarted before scraping!")
                      browser = HeadlessBrowser()
                      FQ_param, PQ_param, DQ_param, ref_urls = browser.visit_url(self.filter)
                      break
              except Exception as e:
                  os.system("pkill -f chromedriver")
                  os.system("pkill -f chrome")
                  n += 1
                  print(f"An error happened in preparing: {e}")
                  time.sleep(2)

            if ref_urls != []:
                self.add_body(FQ_param, PQ_param, DQ_param, ref_urls)

            return FQ_param, PQ_param, DQ_param
        else:
            print("API url is secured.")

            FQ_param = {"url": StoreAPI.filtered_url, "body": None}
            PQ_param = {"url": StoreAPI.preview_url, "body": None}
            DQ_param = {"url": StoreAPI.report_url, "body": None}
            self.add_body(FQ_param, PQ_param, DQ_param, StoreAPI.ref_urls)

            print(FQ_param, PQ_param, DQ_param)

            return FQ_param, PQ_param, DQ_param

    def add_body(self, FQ_param, PQ_param, DQ_param, ref_uls):
        template = {"eventIds":[], "reportIds":[], "countries":[], "firstDiseases":[], "secondDiseases":[], "typeStatuses":[],
                    "reasons":[], "eventStatuses":[], "reportTypes":[], "reportStatuses":[], "eventStartDate":None, "submissionDate":None,
                    "animalTypes":[], "sortColumn":"submissionDate", "sortOrder":"desc", "pageSize":10, "pageNumber":0}
        Country_ID = {ids["name"]: ids for ids in requests.get(ref_uls[0]).json()}
        Disease_ID = {ids["name"]: ids for ids in requests.get(ref_uls[1]).json()}
        Subtype_ID = {ids["name"]: ids for ids in requests.get(ref_uls[2]).json()}
        Status_ID = {ids["translation"]: ids for ids in requests.get(ref_uls[4]).json()}
        Region_ID = {ids["name"]: ids for ids in requests.get(ref_uls[6]).json()}

        for key, value in self.filter.items():
            if value is None:
                continue

            if key in ["country", "region"]:
                for item in value:
                    if key == "country":
                        ids = Country_ID.get(item)
                        if ids: template["countries"].append(ids.get("areaId"))
                    else:
                        ids = Region_ID.get(item)
                        if ids: template["countries"].extend(ids.get("countryIds"))
            elif key == "disease":
                for item in value:
                    for name_key, name_value in Disease_ID.items():
                        if item in name_key:
                            template["firstDiseases"].extend(name_value.get("ids", []))
            elif key == "subtype":
                for item in value:
                    ids = Subtype_ID.get(item)
                    if ids:
                        template["secondDiseases"].extend(ids.get("ids", []))
            elif key == "status":
                for item in value:
                    ids = Status_ID.get(item)
                    if ids:
                        template["eventStatuses"].append(ids.get("keyValue"))

        add_list = [FQ_param, PQ_param, DQ_param]
        for item in add_list:
            if item["body"] == None:
                item["body"] = template


### クラス変数の変更

In [27]:
def update_class_variable(cls, variable_name, value):
    if variable_name in cls.class_variables:
        cls.class_variables[variable_name] = value
    else:
        raise KeyError(f"{variable_name} is not a valid class variable.")

### APIからの情報取得 Ver.3.0


参考 \\
https://youtu.be/et-cDFbVkQw?si=rusBs1kgbXV9V1MK \\
https://qiita.com/tchnkmr/items/b05f321fa315bbce4f77

In [28]:
#import logging

#logging.basicConfig(level=logging.DEBUG, format="%(processName)s: %(message)s")

class CorrectDetail():
    thread_local_data = threading.local()
    data_lock = threading.Lock()
    all_thread_data_D = {}
    record = None

    @classmethod
    def update_classvalues(cls, nowReport, SKIP):
        cls.nowReport = nowReport
        cls.SKIP = SKIP

    @classmethod
    def get_all_thread_data_D(cls):
        with cls.data_lock:
            return cls.all_thread_data_D.copy()

    def __init__(self, df2, from_date, DQ_param):
        self.DQ_param = DQ_param
        self.df2 = df2
        self.from_date = from_date
        self.SR = SendRequest()

    def correct_detaildata(self, eventId, extracted_previewdata, list_number, child_thread_id):
        if (CorrectDetail.record == None) and (CorrectDetail.nowReport != {}):
            CorrectDetail.record = child_thread_id
            number = min(list(CorrectDetail.nowReport.values())[0])
            skip_list_D = CorrectDetail.SKIP[0]
        elif CorrectDetail.nowReport != {}:
            if list_number in list(CorrectSummary.nowList.values()):
                number = min(list(CorrectDetail.nowReport.values())[1])
                skip_list_D = CorrectDetail.SKIP[1]
            else:
                number = 0
                skip_list_D = []
        else:
            number = 0
            skip_list_D = []

        if not self.df2.empty:
            where_eve = self.df2.columns[self.df2.iloc[0] == str(eventId)]
        else:
            where_eve = pd.Index([])

        futures_D = []

        with ThreadPoolExecutor(max_workers=2) as executor:
            while number + 1 <= len(extracted_previewdata):
                if not number in skip_list_D:
                    #logging.debug(f"Thread {executor.name} is running")
                    future = executor.submit(self.process_single_detail, eventId, extracted_previewdata, number, where_eve, list_number, child_thread_id)
                    futures_D.append(future)
                number += 1

            for future in futures_D:
                future.result()

        #CorrectDetail.thread_local_data.nowReport = None

    def process_single_detail(self, eventId, extracted_previewdata, number, where_eve, list_number, child_thread_id):
        thread_id = threading.current_thread().ident
        with CorrectDetail.data_lock:
            CorrectDetail.all_thread_data_D[thread_id] = [number, child_thread_id]
            #CorrectDetail.thread_local_data.nowReport = [number, child_thread_id]

        response = self.SR.send_request_with_cookie(self.DQ_param, 'GET', [eventId, extracted_previewdata[number]])
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            return

        response_data = response.json()
        ref_col, check = self.check_date_and_ref_D(response_data)
        time.sleep(3)

        if check:
            processor1 = Qualitative_DataFormat(response_data, (list_number + number))
            Integrated_Info = processor1.integrate_information(eventId)

            with CorrectDetail.data_lock:
                with ProcessPoolExecutor(max_workers=2) as process_executor:  # マルチプロセス化
                    future2 = process_executor.submit(ControlDataFormat, response_data, self.df2, (list_number + number))
                    future3 = process_executor.submit(QuantitativeDataFormat, response_data, self.df2, (list_number + number))

                Control_content, df2_first = future2.result().get_control_content()
                Quant_data, df2_latter = future3.result().update_ws_quant_data()

                self.df2 = pd.concat([df2_first, df2_latter], axis=0).reset_index(drop=True)
                Data = pd.concat([Integrated_Info, Control_content, Quant_data], axis=0).astype(str).reset_index(drop=True)

                if ref_col is not None:
                    self.df2[ref_col] = Data['value'] if (list_number + number) == 0 else Data
                else:
                    if not where_eve.empty:
                        new_row_col = int(where_eve[-1]) + 1
                        self.df2 = pd.concat([self.df2.iloc[:, :new_row_col], Data, self.df2.iloc[:, new_row_col:]], axis=1, ignore_index=True).reset_index(drop=True)
                    else:
                        self.df2 = pd.concat([self.df2, Data], axis=1, ignore_index=True).reset_index(drop=True)

        #print(f"Get information from report ID: {extracted_previewdata[number]}!")

    def check_date_and_ref_D(self, item):
        outbreak = pd.json_normalize(item['outbreak']).iloc[0]
        outbreakId = outbreak['outbreakId']
        Enddate = pd.to_datetime(outbreak['endDate']).date().strftime('%Y-%m-%d') if outbreak['endDate'] else '-'

        if not self.df2.empty:
            where_ref = self.df2.columns[self.df2.iloc[1] == str(outbreakId)]
        else:
            where_ref = pd.Index([])

        if self.from_date is not None:
            date_list = [Enddate, self.from_date]
            check = self.CD.narrow_down(date_list)
        else:
            check = False

        return where_ref[0] if not where_ref.empty else None, (not check)


In [29]:
class CorrectSummary(CorrectDetail):
    #thread_local_data_S = threading.local()
    lock_S = threading.Lock()
    all_thread_data_S = {}
    pbar_lock = threading.Lock()
    #skip_num = 0

    @classmethod
    def update_classvalues(cls, nowList, skip):
        cls.nowList = nowList
        cls.skip_list = skip

    @classmethod
    def get_all_thread_data_S(cls):
        with cls.data_lock_S:
            return cls.all_thread_data_S.copy()

    def __init__(self, df1, df2, from_date, PQ_param, DQ_param):
        self.PQ_param = PQ_param
        self.df1 = df1
        self.from_date = from_date
        self.SR = SendRequest()
        super().__init__(df2, from_date, DQ_param)

    def correct_previewdata(self, extracted_listdata):
        number = min(list(CorrectSummary.nowList.values())) if CorrectSummary.nowList != {} else 0
        total_tasks = len(extracted_listdata) - len(CorrectSummary.skip_list)

        with tqdm(total=total_tasks, desc="Processing previews") as pbar:
            futures = []

            with ThreadPoolExecutor(max_workers=2) as executor:
                while number + 1 <= len(extracted_listdata):
                    if not (number in CorrectSummary.skip_list):
                        future = executor.submit(self.process_single_preview, extracted_listdata, number, pbar)
                        futures.append(future)
                    number += 1

            for future in futures:
                future.result()

        pbar.close()

        #if CorrectSummary.nowList == {}:
        #    total_events = total_tasks - CorrectSummary.skip_num
        #    print("Total events: f{total_events}")
        #else:
        #    added_events = total_tasks - CorrectSummary.skip_num
        #    print("Added events: f{added_events}")

    def process_single_preview(self, extracted_listdata, number, pbar):
        parent_thread_id = threading.current_thread().ident
        #print(parent_thread_id)
        with CorrectSummary.lock_S:
            CorrectSummary.all_thread_data_S[parent_thread_id] = number
            #CorrectSummary.thread_local_data_S.nowList = number

        response = self.SR.send_request_with_cookie(self.PQ_param, 'GET', [extracted_listdata[number]['eventId']])
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            return

        response_data = response.json()
        ref_col, check = self.check_date_and_ref_S(response_data)

        if check:
            processor1 = Qualitative_DataFormat(response_data, number)
            Integrated_Info = processor1.integrate_information()

            with CorrectSummary.lock_S:
                with ProcessPoolExecutor(max_workers=2) as process_executor:
                    future2 = process_executor.submit(ControlDataFormat, response_data, self.df1, number)
                    future3 = process_executor.submit(QuantitativeDataFormat, response_data, self.df1, number)

                Control_content, df1_first = future2.result().get_control_content()
                Quant_data, df1_latter = future3.result().update_ws_quant_data()

                self.df1 = pd.concat([df1_first, df1_latter], axis=0).reset_index(drop=True)
                Data = pd.concat([Integrated_Info, Control_content, Quant_data], axis=0).astype(str).reset_index(drop=True)

                if ref_col is not None:
                    self.df1[ref_col] = Data['value'] if number == 0 else Data
                else:
                    self.df1 = pd.concat([self.df1, Data], axis=1, ignore_index=True).reset_index(drop=True)
        #else:
        #    with CorrectSummary.lock_S:
        #        CorrectSummary.skip_num += 1

        outbreak_df = pd.json_normalize(response_data['outbreaks'])
        if not outbreak_df.empty:
            outbreak_ids = outbreak_df['outbreakId'].tolist()
            self.correct_detaildata(extracted_listdata[number]['eventId'], outbreak_ids, number, parent_thread_id)
        self.df1.to_csv('Summary.csv', index=False)
        self.df2.to_csv('Detail.csv', index=False)

        #print(f"Get all report in list ID: {extracted_listdata[number]}!")

        with CorrectSummary.pbar_lock:
            pbar.update(1)


    def check_date_and_ref_S(self, item):
        events = pd.json_normalize(item['event']).iloc[0]
        eventId = events['eventId']
        Enddate = pd.to_datetime(events['endedOn']).date().strftime('%Y-%m-%d') if events['endedOn'] else '-'

        if not self.df1.empty:
            where_ref = self.df1.columns[self.df1.iloc[0] == str(eventId)]
        else:
            where_ref = pd.Index([])

        if self.from_date is not None:
            date_list = [Enddate, self.from_date]
            check = self.CD.narrow_down(date_list)
        else:
            check = False

        return where_ref[0] if not where_ref.empty else None, (not check)


In [30]:
class CorrectList(CorrectSummary):
    @classmethod
    def update_classvalues(cls, nowPage=None, finished=False, extracted_listdata=[]):
        cls.nowPage = nowPage
        cls.finished = finished
        cls.extracted_listdata = extracted_listdata

    def __init__(self, df1, df2, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param):
        self.FQ_param = FQ_param
        self.to_date = to_date
        self.CD = CompareDate()
        self.SR = SendRequest()
        self.pre_date = previous_date
        self.df1 = df1
        self.df2 = df2
        super().__init__(df1, df2, from_date, PQ_param, DQ_param)

    def scrape(self):
        if not CorrectList.finished:
            self.correct_listdata()
        self.correct_previewdata(CorrectList.extracted_listdata)
        self.final_sort(1, [0], [False])
        self.final_sort(2, [0, 1], [False, False])


    def correct_listdata(self):
        # リクエストを送信し、結果を表示
        if CorrectList.nowPage:
            self.FQ_param["body"]["pageNumber"] = CorrectList.nowPage
        else:
            CorrectList.nowPage = 0

        while True:
            #print(CorrectList.nowPage)

            response = self.SR.send_request_with_cookie(self.FQ_param, 'POST')
            if response.status_code != 200:
                print(f"Error: Received status code {response.status_code}")
                break

            response_data = response.json()

            if not response_data.get('list'):
                print("All list IDs are obtained.")
                CorrectList.finished = True
                break

            # 必要な情報を抽出してリストに整理
            for item in response_data['list']:
                check = self.check_date(item)

                if check:
                    CorrectList.extracted_listdata.append({
                        'eventId': item['eventId']
                    })

            # 次のページへ
            self.FQ_param["body"]["pageNumber"] += 1
            CorrectList.nowPage += 1

    def check_date(self, item):
        Update_date = pd.to_datetime(item['submissionDate']).date() if item['submissionDate'] is not None else '-'
        Start_date = pd.to_datetime(item['eventStartDate']).date() if item['eventStartDate'] is not None else '-'

        if self.pre_date is not None:
            date_list = [Update_date, self.pre_date]
            checkA = self.CD.narrow_down(date_list)
        else:
            checkA = False

        if self.to_date is not None:
            date_list2 = [self.to_date, Start_date]
            checkB = self.CD.narrow_down(date_list2)
        else:
            checkB = False

        return not (checkA or checkB)

    def final_sort(self, number, columns, ascending_list):
        if number == 1:
            df = self.df1
            name = "Summary"
        elif number == 2:
            df = self.df2
            name = "Detail"

        df = df.T.reset_index()
        df_subset = df.iloc[3:, :]
        df_subset[columns] = df_subset[columns].astype(int)
        df_sorted_subset = df_subset.sort_values(by=columns, ascending=ascending_list)
        df.iloc[3:, :] = df_sorted_subset
        df[columns] = df[columns].astype(str)
        df.drop(columns=['index'], axis=1).reset_index().to_csv(f'{name}_sorted.csv', index=False)


### データ整理クラス関数 ver.2.5

In [31]:
class Qualitative_DataFormat:
    def __init__(self, response_data, number):
        self.response_data = response_data
        self.number = number

    def create_common_info_S(self):
        events = self.response_data['event']
        sources = pd.json_normalize(self.response_data['sources'])
        methods = pd.json_normalize(self.response_data['methods'])
        laboratoryTests = pd.json_normalize(self.response_data['laboratoryTests'])

        common_info = {
            'eventId': events['eventId'],
            'country.areaId': events['country']['areaId'],
            'country.name': events['country']['name'],
            'country.isoCode': events['country']['isoCode'],
            'startedOn': pd.to_datetime(events['startedOn']).date() if events['startedOn'] else '-',
            'lastOccurrence': pd.to_datetime(events['lastOccurrence']).date() if events['lastOccurrence'] else '-',
            'endedOn': pd.to_datetime(events['endedOn']).date() if events['endedOn'] else '-',
            'eventStatus.id': events['eventStatus']['id'],
            'eventStatus.translation': events['eventStatus']['translation'],
            'isAquatic': events['isAquatic'],
            'disease.diseaseId': events['disease']['diseaseId'],
            'disease.name': events['disease']['name'],
            'subType.diseaseId': events['subType']['disease']['diseaseId'] if events['subType']['disease'] else 'Pending',
            'subType.name': events['subType']['disease']['name'] if events['subType']['disease'] else 'Pending',
            'reason.id': events['reason']['id'],
            'reason.translation': events['reason']['translation'],
            'clinicalSigns': events['clinicalSigns'],
            'sources.sourceId': sources.get('source.sourceId', pd.Series(['-'])).tolist(),
            'sources.name': sources.get('source.name', sources.get('otherSource.original', pd.Series(['-']))).tolist(),
            'eventComment': events['eventComment']['translation'] if events['eventComment'] else 'No epidemiological comment',
            'methods.id': methods['nature.id'].tolist(),
            'methods.translation': methods['nature.translation'].tolist(),
            'laboratoryTests.testName': laboratoryTests['testName'].tolist() if not laboratoryTests.empty else '-',
            'confirmOn': pd.to_datetime(events['confirmOn']).date() if events['confirmOn'] else '-',
            'laboratoryTests.result': laboratoryTests['result'].tolist() if not laboratoryTests.empty else '-',
        }

        return common_info

    def create_common_info_D(self, eventId):
        outbreak = pd.json_normalize(self.response_data['outbreak']).iloc[0]
        adminDivisions = self.response_data['adminDivisions']
        diagnosticMethods = pd.json_normalize(self.response_data['diagnosticMethods'])

        common_info = {
            'eventId': eventId,
            'outbreakId': outbreak['outbreakId'],
            'areaId': outbreak['areaId'],
            'latitude': outbreak['latitude'],
            'longtitude': outbreak['longitude'],
            'epiUnitType_id': outbreak['epiUnitType.id'],
            'epiUnitType_translation': outbreak['epiUnitType.translation'],
            'startDate': pd.to_datetime(outbreak['startDate']).date() if outbreak['startDate'] else '-',
            'endDate': pd.to_datetime(outbreak['endDate']).date() if outbreak['endDate'] else '-',
            'diagnosticMethods.id': diagnosticMethods['nature.id'].tolist() if not diagnosticMethods.empty else '-',
            'diagnosticMethods.translation': diagnosticMethods['nature.translation'].tolist() if not diagnosticMethods.empty else '-',
        }

        # 管理区分の情報を初期化し、存在する場合は更新
        for i in range(3):
            common_info[f'adminDivisions.{i+1}.id'] = '-'
            common_info[f'adminDivisions.{i+1}.name'] = '-'
        for item in adminDivisions:
            L = item['adminLevel']
            common_info[f'adminDivisions.{L}.id'] = item['areaId']
            common_info[f'adminDivisions.{L}.name'] = item['name']

        return common_info

    def integrate_information(self, eventId=None):
        if 'event' in self.response_data:
            info = self.create_common_info_S()
        elif 'outbreak' in self.response_data:
            info = self.create_common_info_D(eventId)

        df = pd.DataFrame({
            'Item': ['Summary'] * len(info),
            'species': [''] * len(info),
            'category': list(info.keys()),
            'value': list(info.values())
        })

        if self.number == 0:
            Integrated_Info = df
        else:
            Integrated_Info = df[['value']]

        return Integrated_Info


In [32]:
class ControlDataFormat:
    def __init__(self, response_data, df, number):
        self.response_data = response_data
        self.df = df
        self.number = number

    def extract_measures_info_S(self, measures_data):
        measures_name = []
        domestic_value = []
        wild_value = []

        for index, measure in measures_data.iterrows():
            if measure['measure.name'] not in measures_name:
                measures_name.append(measure['measure.name'])

                domestic_status = '-'
                wild_status = '-'

                category_original = measure['measure.category.original'].lower()
                if 'domestic' in category_original:
                    domestic_status = measure['status.translation']
                elif 'wild' in category_original:
                    wild_status = measure['status.translation']

                domestic_value.append(domestic_status)
                wild_value.append(wild_status)

        df = pd.DataFrame()
        df['Item'] = ['Control Measure'] * len(measures_name) * 2
        df['species'] = [item for item in ['Domestic', 'Wild'] for _ in range(len(measures_name))]
        df['category'] = measures_name * 2
        df['value'] = domestic_value + wild_value
        report_cont = list(dict.fromkeys(measures_name))

        return df, report_cont

    def extract_measures_info_D(self):
        measure_name = []
        measure_apply = []
        measure = self.response_data['controlMeasures']
        add_measures = self.response_data['additionalMeasures']

        for item in measure:
            if item['measure']['name'] not in measure_name:
                measure_name.append(item['measure']['name'])

                default_status = '-'
                if item['status']['translation'] == 'Applied':
                    measure_apply.append('Applied')
                else:
                    measure_apply.append(default_status)

        for item in add_measures:
            if item['name'] not in measure_name:
                measure_name.append(item['name'])
                measure_apply.append('Applied')

        df = pd.DataFrame()
        df['Item'] = ['Control Measure'] * len(measure_apply)
        df['species'] = ['' for _ in range(len(measure_apply))]
        df['category'] = measure_name
        df['value'] = measure_apply
        report_cont = list(dict.fromkeys(df['category']))

        return df, report_cont

    def get_cont_unique(self):
        if not self.df.empty:
            cont_unique = self.df[self.df[0] == 'Control Measure'][2].unique().tolist()
        else:
            cont_unique = []
        return cont_unique

    def update_df_cont_data(self, cont_unique, report_cont, control_df):

        order, order_name, record = self.get_order_and_record(cont_unique, report_cont, control_df)

        Cont_df = self.get_after_cont_content(order, order_name, record, report_cont, cont_unique, control_df)

        Cont_data = self.add_missing_species(Cont_df, cont_unique, order)

        return Cont_data

    def get_order_and_record(self, cont_unique, report_cont, control_df):
        order = []
        order_name = []
        record = []
        insert_ver = True if cont_unique == [] else False

        for mea in report_cont:
            try:
                order.append(cont_unique.index(mea))
                order_name.append(mea)
            except ValueError:
                record.append(mea)
                self.insert_new_cols(mea, control_df, insert_ver)
                insert_ver = False

        return order, order_name, record

    def insert_new_cols(self, cont_category, df, insert_ver):
        new_rows = df[df['category'] == cont_category].drop('value', axis=1).astype(str)
        new_rows.columns = [0, 1, 2]
        if insert_ver:
            summary_row_index = self.df[self.df[0] == 'Summary'].index.values[-1] + 1
            self.df = pd.concat([self.df.iloc[:summary_row_index], new_rows, self.df.iloc[summary_row_index:]]).reset_index(drop=True)
        else:
            control_measure_row_index = self.df[self.df[0] == 'Control Measure'].index.values

            if 'event' in self.response_data.keys():
                domestic = new_rows[new_rows[1] == 'Domestic']
                wild = new_rows[new_rows[1] == 'Wild']

                domestic_row_index = control_measure_row_index[(len(control_measure_row_index)//2)]
                self.df = pd.concat([self.df.iloc[:domestic_row_index], domestic, self.df.iloc[domestic_row_index:]], ignore_index=True)

                wild_insert_row_index = control_measure_row_index[-1] + len(domestic) + 1
                self.df = pd.concat([self.df.iloc[:wild_insert_row_index], wild, self.df.iloc[wild_insert_row_index:]], ignore_index=True)
            else:
                self.df = pd.concat([self.df.iloc[:control_measure_row_index[-1]+1], new_rows, self.df.iloc[control_measure_row_index[-1]+1:]], ignore_index=True)



    def get_after_cont_content(self, order, order_name, record, report_cont, cont_unique, control_df):
        order_sorted = sorted(order)
        after_Cont_content = pd.DataFrame()
        if order != order_sorted:
            for i in order_sorted:
                index = order.index(i)
                df_to_append = control_df[control_df['category'] == order_name[index]]
                after_Cont_content = pd.concat([after_Cont_content, df_to_append], ignore_index=True)
        else:
            for i in range(len(order)):
                df_to_append = control_df[control_df['category'] == order_name[i]]
                after_Cont_content = pd.concat([after_Cont_content, df_to_append], ignore_index=True)

        if len(record) != 0:
            for name2 in record:
                df_to_append2 = control_df[control_df['category'] == name2]
                after_Cont_content = pd.concat([after_Cont_content, df_to_append2], ignore_index=True)

        after_Cont_content = after_Cont_content.reindex(columns=['Item', 'species', 'category', 'value'])
        return after_Cont_content

    def add_missing_species(self, Cont_df, cont_unique, order):
        if len(cont_unique) != len(order):
            Cont_list = Cont_df['value'].astype(str).tolist()
            diff_set = set([h for h in range(len(cont_unique))]) - set(order)
            missing = sorted([diff for diff in diff_set])

            if Cont_list == [] and 'event' in self.response_data.keys():
                Cont_list = np.full(len(missing) * 2, '-', dtype=object)
            elif Cont_list == []:
                Cont_list = np.full(len(missing), '-', dtype=object)
            else:
                if 'measures' in list(self.response_data.keys()):
                    instant = np.reshape(Cont_list, (2, -1))
                    for c in range(len(missing)):
                        if missing[c] < instant.shape[1]:
                            instant = np.insert(instant, missing[c], np.array(['-' for _ in range(2)], dtype=object), axis=1)
                        else:
                            instant = np.insert(instant, instant.shape[1], np.array(['-' for _ in range(2)], dtype=object), axis=1)
                    Cont_list = instant.ravel()
                else:
                    instant = np.reshape(Cont_list, (1, -1))
                    for c in range(len(missing)):
                        if missing[c] < instant.shape[0]:
                            instant = np.insert(instant, missing[c], '-')
                        else:
                            instant = np.insert(instant, instant.shape[0], '-')
                    Cont_list = instant.ravel()

            return pd.DataFrame(Cont_list.tolist(), columns=['value'])
        else:
            return Cont_df.filter(items=['value'])

    def get_control_content(self):
        if 'measures' in list(self.response_data.keys()):
            measures_data = pd.json_normalize(self.response_data['measures'])
            measures_df, report_cont = self.extract_measures_info_S(measures_data)
        else:
            measures_df, report_cont = self.extract_measures_info_D()

        cont_unique = self.get_cont_unique()

        if self.number == 0:
            Control_content = measures_df
        elif cont_unique != report_cont:
            Control_content = self.update_df_cont_data(cont_unique, report_cont, measures_df)
        else:
            Control_content = measures_df[['value']]


        if (not self.df.empty) and (self.df[0].isin(['Control Measure'])).any():
            control_measure_row_index = self.df[self.df[0] == 'Control Measure'].index.values[-1] + 1
            self.df = self.df.iloc[:control_measure_row_index]
        elif (not self.df.empty) and (self.df[0].isin(['Summary'])).any():
            summary_row_index = self.df[self.df[0] == 'Summary'].index.values[-1] + 1
            self.df = self.df.iloc[:summary_row_index]
        else:
            self.df = pd.DataFrame()

        return Control_content, self.df


In [33]:
class QuantitativeDataFormat:
    def __init__(self, response_data, df, number):
        self.response_data = response_data
        self.df = df
        self.number = number

    def preprocessing(self, p_type):
        if p_type == 'Summary':
            quantitative_data = self.response_data['quantitativeData']
            news_data = quantitative_data.get('news', [])
            totals_data = quantitative_data.get('totals', [])
            unit_data = quantitative_data.get('unit', {}) or {}
        elif p_type == 'Detail':
            unit_data = self.response_data['quantityUnit']
            quantitaive_data = self.response_data['speciesQuantities']
            news_data, totals_data = [], []
            for item in quantitaive_data:
                nd = item.get('newQuantities', None)
                td = item.get('totalQuantities', None)
                if nd:
                    news_data.append(nd)
                if td:
                    totals_data.append(td)

        return news_data, totals_data, unit_data

    def create_quantitative_dataframe(self, p_type):
        news_data, total_data, unit_data = self.preprocessing(p_type)

        species_dict = {}

        # Process 'news' data
        for item in news_data:
            species_type = f"{item['speciesName']} (Domestic)" if not item['isWild'] else f"{item['speciesName']} (Wild)"
            if species_type not in species_dict:
                species_dict[species_type] = {}
            species_dict[species_type]['news'] = {
                'New-Susceptible': item.get('susceptible', '-') or '-',
                'New-Cases': item.get('cases', '-') or '-',
                'New-Deaths': item.get('deaths', '-') or '-',
                'New-Killed': item.get('killed', '-') or '-',
                'New-Slaughtered': item.get('slaughtered', '-') or '-',
                'New-Vaccinated': item.get('vaccinated', '-') or '-'
            }

        # Process 'totals' data
        for item in total_data:
            species_type = f"{item['speciesName']} (Domestic)" if not item['isWild'] else f"{item['speciesName']} (Wild)"
            if species_type not in species_dict:
                species_dict[species_type] = {}
            species_dict[species_type]['totals'] = {
                'Total-Susceptible': item.get('susceptible', '-') or '-',
                'Total-Cases': item.get('cases', '-') or '-',
                'Total-Deaths': item.get('deaths', '-') or '-',
                'Total-Killed': item.get('killed', '-') or '-',
                'Total-Slaughtered': item.get('slaughtered', '-') or '-',
                'Total-Vaccinated': item.get('vaccinated', '-') or '-'
            }

        species_list = []
        category_list = []
        value_list = []
        for species, data in species_dict.items():
            species_list.extend([species]*12)
            category_list.extend(data.get('news', {}).keys() or ['New-Susceptible', 'New-Cases', 'New-Deaths', 'New-Killed', 'New-Slaughtered', 'New-Vaccinated'])
            value_list.extend(data.get('news', {}).values() or ['-', '-', '-', '-', '-', '-'])
            category_list.extend(data.get('totals', {}).keys() or ['Total-Susceptible', 'Total-Cases', 'Total-Deaths', 'Total-Killed', 'Total-Slaughtered', 'Total-Vaccinated'])
            value_list.extend(data.get('totals', {}).values() or ['-', '-', '-', '-', '-', '-'])

        # Create DataFrame
        df = pd.DataFrame({
            'Item': ['Quantitative']*len(species_list),
            'species': species_list,
            'category': category_list,
            'value': value_list
        })

        return unit_data.get('id', '-'), unit_data.get('translation', '-'), df

    def get_species_unique(self):
        if not self.df.empty:
            draft = self.df[self.df[0] == 'Quantitative'][1].unique().tolist()
            species_unique = [x for x in draft if x != "" and not pd.isna(x)]
        else:
            species_unique = []
        return species_unique

    def _reorder_quant_values(self, quantitaive_df, report_species, species_unique):
        order, order_name, record = self._get_order_and_record(report_species, species_unique, quantitaive_df)

        after_Quant_value = self._get_after_quant_values(order, order_name, record, quantitaive_df)

        Quant_data = self._add_missing_species(after_Quant_value, species_unique, order)

        return Quant_data

    def _get_order_and_record(self, report_species, species_unique, quantitaive_df):
        order = []
        order_name = []
        record = []
        for species in report_species:
            try:
                order.append(species_unique.index(species))
                order_name.append(species)
            except ValueError:
                record.append(species)
                self.insert_new_rows(species, quantitaive_df)
        return order, order_name, record

    def _get_after_quant_values(self, order, order_name, record, quantitaive_df):
        order_sorted = sorted(order)
        after_Quant_value = pd.DataFrame()
        if order != order_sorted:
            for i in order_sorted:
                index = order.index(i)
                df_to_append = quantitaive_df[quantitaive_df['species'] == order_name[index]]
                after_Quant_value = pd.concat([after_Quant_value, df_to_append], ignore_index=True)
        else:
            for i in range(len(order)):
                df_to_append = quantitaive_df[quantitaive_df['species'] == order_name[i]]
                after_Quant_value = pd.concat([after_Quant_value, df_to_append], ignore_index=True)

        if record:
            for name2 in record:
                df_to_append2 = quantitaive_df[quantitaive_df['species'] == name2]
                after_Quant_value = pd.concat([after_Quant_value, df_to_append2], ignore_index=True)

        after_Quant_value = after_Quant_value.reindex(columns=['Item', 'species', 'category', 'value'])
        return after_Quant_value

    def _add_missing_species(self, Quant_df, species_unique, order):
        Qunat_list = Quant_df['value'].astype(str).tolist()
        diff_set = set([h for h in range(len(species_unique))]) - set(order)
        missing = sorted(diff for diff in diff_set)
        for c in range(len(missing)):
            Qunat_list[(12*missing[c]):(12*missing[c])] = ['-' for _ in range(12)]
        Quant_data = pd.DataFrame(Qunat_list, columns=['value'])
        return Quant_data

    def insert_new_rows(self, species, df):
        new_row = df[df['species'] == species].drop('value', axis=1).astype(str)
        new_row.columns = [0, 1, 2]
        qua_row_index = self.df[self.df[0] == 'Quantitative'].index.values
        self.df = pd.concat([self.df.iloc[:qua_row_index[-1] + 1], new_row, self.df.iloc[qua_row_index[-1] + 1:]]).reset_index(drop=True)

    def update_ws_quant_data(self):
        if 'quantitativeData' in self.response_data:
            unitId, unitName, quantitaive_df = self.create_quantitative_dataframe("Summary")
        else:
            unitId, unitName, quantitaive_df = self.create_quantitative_dataframe("Detail")

        species_unique = self.get_species_unique()
        report_species = quantitaive_df['species'].unique().tolist()

        unit_data = {
            'Item': ['Quantitative', 'Quantitative'],
            'species': ['', ''],
            'category': ['id', 'Name'],
            'value': [unitId, unitName]
        }
        unit_df = pd.DataFrame(unit_data)


        if self.number == 0:
            Quant_data = pd.concat([unit_df, quantitaive_df], axis=0)
        elif species_unique != report_species:
            Quant_data = self._reorder_quant_values(quantitaive_df, report_species, species_unique)
            Quant_data = pd.concat([unit_df[['value']], Quant_data], axis=0)
        else:
            Quant_data = pd.concat([unit_df[['value']], quantitaive_df[['value']]], axis=0)

        if not self.df.empty:
            qua_row_index = self.df[self.df[0] == 'Quantitative'].index.values
            self.df = self.df.iloc[qua_row_index[0]:qua_row_index[-1] + 1]
        else:
            self.df = pd.DataFrame()

        return Quant_data, self.df


### 統合 Ver.3

参考 \\
https://qiita.com/puchi2121/items/3983a75c8c0a6e958fbd

In [34]:
class AutoReportScraper(CorrectList):
    def __init__(self, df1, df2, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param):
        self.df1 = df1
        self.df2 = df2
        super().__init__(df1, df2, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param)

    def scrape_reports_with_retry(self):
        n = 0
        prev_list = []
        same = 0
        while True:
          try:
              if n == 0:
                self.scrape()
              else:
                print("Restarted!")
                self.scrape()
          except Exception as e:
              n += 1
              print(f"An error happened: {e}")
              self.df1.to_csv('Summary.csv', index=False)
              self.df2.to_csv('Detail.csv', index=False)
              #print(self.df1)
              #print(self.df2)
              etype, value, tb = sys.exc_info()
              print(traceback.format_exception(etype, value, tb))
              # エラーが発生した場合、現在の状態を取得して再実行する
              now_list = [CorrectList.nowPage, CorrectSummary.all_thread_data_S, CorrectDetail.all_thread_data_D]
              print(now_list)
              self._get_current_state(CorrectList.nowPage, CorrectSummary.all_thread_data_S, CorrectDetail.all_thread_data_D, CorrectList.finished, CorrectList.extracted_listdata)
              if (prev_list != []) and (prev_list == now_list):
                same += 1
                if same >= 5:
                  break
              if "429" in str(e):
                print("waiting 10 seconds ...")
                time.sleep(10)
                same -= 1
              prev_list = now_list
              time.sleep(2)
          else:
              self.df1.to_csv('Summary.csv', index=False)
              #print(self.df1)
              self.df2.to_csv('Detail.csv', index=False)
              #print(self.df2)
              break


    def _get_current_state(self, Page, List, Report, Finished, eventId_list):
        CorrectList.nowPage = Page
        MS = MakingSkip(List, Report)
        List_skip = MS.List_making()
        Report_skip = MS.Report_maikng()
        CorrectSummary.nowList = List
        CorrectSummary.skip_list = List_skip
        CorrectDetail.nowReport = Report
        CorrectDetail.SKIP = Report_skip
        CorrectList.finished = Finished
        CorrectList.extracted_listdata = eventId_list

In [35]:
class ScrapingWAHIS():
    def __init__(self, disease, Event_status = None, Country = None, World_region = None, SubType = None,
                 Summary = None, Detail = None, previous_date = None, from_date = None, to_date = None,
                 nowPage = None, nowList = {}, nowReport = {}):
        self.filter = {"disease": disease,
                       "status": Event_status,
                       "country": Country,
                       "region": World_region,
                       "subtype": SubType}

        if Summary is not None:
            self.df1 = pd.read_csv(Summary)
            self.df1.columns = self.df1.columns.astype(int)
        else:
            self.df1 = pd.DataFrame()

        if Detail is not None:
            self.df2 = pd.read_csv(Detail)
            self.df2.columns = self.df2.columns.astype(int)
        else:
            self.df2 = pd.DataFrame()

        self.pre_date = datetime.strptime(previous_date, '%Y/%m/%d').date() if previous_date is not None else None
        self.fd = datetime.strptime(from_date, '%Y/%m/%d').date() if from_date is not None else None
        self.td = datetime.strptime(to_date, '%Y/%m/%d').date() if to_date is not None else None

        MS = MakingSkip(nowList, nowReport)
        List_skip = MS.List_making()
        Report_skip = MS.Report_maikng()

        CorrectList.update_classvalues(nowPage)
        CorrectSummary.update_classvalues(nowList, List_skip)
        CorrectDetail.update_classvalues(nowReport, Report_skip)


    #@retry(retry=retry_if_exception_type((TimeoutError, APIError, HTTPError)))
    def run_scraping(self):
        SA = StoreAPI(self.filter)
        FQ_param, PQ_param, DQ_param = SA.check_API()


        print("start")

        ARS = AutoReportScraper(self.df1, self.df2, self.pre_date, self.fd, self.td, FQ_param, PQ_param, DQ_param)
        ARS.scrape_reports_with_retry()

In [36]:
class MakingSkip():
    def __init__(self, List_progress = {}, Report_progress = {}):
        self.List = list(List_progress.values()) if List_progress != {} else None
        self.Report = list(Report_progress.values()) if Report_progress != {} else None
        self.ReportKey = list(Report_progress.keys()) if Report_progress != {} else None

    def List_making(self):
        if self.List == None:
            return []
        else:
            max_value = max(self.List)
            skip_list = set([h for h in range(max_value)]) - set(self.List)
            return skip_list

    def Report_maikng(self):
        if self.Report == None:
            return []
        else:
            arrange = {}
            for value, thread_id in self.Report:
                if thread_id not in arrange:
                    arrange[thread_id] = []
                arrange[thread_id].append(value)
            each_progress = list(arrange.values())

            SKIP = {}
            for i, thread_progress in enumerate(each_progress):
                ID = self.ReportKey[i]
                max_value_D = max(thread_progress)
                skip_list_D = set([h for h in range(max_value_D)]) - set(thread_progress)
                SKIP[ID] = skip_list_D
            return list(SKIP.values())

# 実行部分（メイン・展開してください）

説明下方

In [37]:
# disease, Event_status, Country, World_region, SubType : ["XXXX", "YYYY", ...]
# Summary, Detail : "/content/XXXXXX"
# previous_date, from_date, to_date : "YYYY/MM/DD"
# nowPage : int
# nowList, nowReport : {XXXXXX: YYYY, PPPPP: QQQQQ}
SW = ScrapingWAHIS(disease = ["Foot and mouth disease", "Peste des"], Event_status = ["Resolved"], Country = None, World_region = ["Asia"], SubType = ["O"],
                   Summary = None, Detail = None,
                   previous_date = None, from_date = "2015/1/1", to_date = "2023/1/1",
                   nowPage = None,
                   nowList =  {},
                   nowReport = {})
SW.run_scraping()

API url is secured.
{'url': 'https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en', 'body': {'eventIds': [], 'reportIds': [], 'countries': [2, 10, 17, 24, 22, 34, 33, 123, 43, 233, 57, 87, 102, 109, 108, 112, 113, 115, 119, 118, 120, 186, 126, 127, 122, 128, 129, 139, 161, 144, 153, 151, 172, 176, 177, 189, 181, 191, 196, 200, 134, 219, 224, 223, 227, 231, 226, 8, 240, 246, 250], 'firstDiseases': [437, 805], 'secondDiseases': [793], 'typeStatuses': [], 'reasons': [], 'eventStatuses': ['resolved'], 'reportTypes': [], 'reportStatuses': [], 'eventStartDate': None, 'submissionDate': None, 'animalTypes': [], 'sortColumn': 'submissionDate', 'sortOrder': 'desc', 'pageSize': 10, 'pageNumber': 0}} {'url': 'https://wahis.woah.org/api/v1/pi/review/event/0000/all-information?language=en', 'body': {'eventIds': [], 'reportIds': [], 'countries': [2, 10, 17, 24, 22, 34, 33, 123, 43, 233, 57, 87, 102, 109, 108, 112, 113, 115, 119, 118, 120, 186, 126, 127, 122, 128, 129, 139, 161, 144, 153, 

Processing previews: 100%|██████████| 117/117 [25:54<00:00, 13.28s/it]
<ipython-input-30-b79e6e007f1b>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[columns] = df_subset[columns].astype(int)
<ipython-input-30-b79e6e007f1b>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[columns] = df_subset[columns].astype(int)


#### 実行中断時、接続を切る前に必ず実行してください

In [20]:
#スクレイピングを中断する場合、接毒解除する前に実行してください
print([CorrectList.nowPage, CorrectSummary.all_thread_data_S, CorrectDetail.all_thread_data_D])

[41, {135143251420736: 28, 135143335347776: 26}, {135143360525888: [0, 135143251420736], 135143410882112: [1, 135143251420736], 135143368918592: [7, 135143335347776], 135143352133184: [6, 135143335347776]}]


## **使い方及び出力ファイルについて**



### **概要**

このコードは[WAHISデータベース](https://wahis.woah.org/#/event-management)のスクレイピングを目的としています。

上記データベースではWAHIS（旧OIE）に世界各地の動物の感染症の報告が登録されていて、国レベルでの発生に関する各リストの中にさらに詳細な地域レベルでのレポートが多数含まれています。
ただし上のサイトで情報を出力しようとするとリストの概要までしか取得できず、個々のレポートはPDF単位でしか入手できないため、情報の活用がしにくいと思い本コードを作成しました。

元々は卒論に使うデータの収集用に作っていましたが、テーマ変更に伴い使用することがなくなりました。しかし「どうせ作ったので共有して使ってもらった方が良いか」と思い公開しています。独学で作ってますので非効率的な部分などあるかと思いますし、自身のドライブにコピーして改変などしていただく分には全く問題ございません。

### **使い方**

「環境構築」及び「スクレイピング関数」の部分を実行してもらった後は、基本的にGoogle colaboratoryの接続が切れない限りは「実行部分」のみの実行だけで問題ないです。

"start"が表示されるとスクレイピングの第一段階（APIのためのID取得）が始まります。

"All list IDs are obtained."が表示されるとスクレイピングの第二段階（APIを用いての情報収集）が始まります。

第二段階の進捗についてはプログレスバーで確認できます。

スクレイピングが完了すると、ページ左端の📁マークを開いてもらえれば"Summary_sorted.csv", "Detail_sorted.csv"が取得できます。Summaryにリストレベル、Detailにレポートレベルでの情報が格納されています。なおソート及び転置はされていませんが、スクレイピングの途中でも逐次データを保存するようにしているので、実行途中で止めてデータをダウンロードしておくことも可能です。

#### **注意点**


*   WAHISデータベースのAPIを使用してスクレイピングをする造りですが、APIが変わった場合を考慮して念のために最初にseleniumを用いてブラウザ（非表示）を立ち上げでデータベースにアクセスしています。その関係上、使用環境及び状況などによってブラウザの立ち上げがうまくいかず、そもそもスクレイピングが始まらないことがあります。

*   ネットワークへの接続およびデータベースへのアクセスに関しては何度か試し、アクセスが難しそうであれば仕切り直して再度接続し直すようにコードを組んであります。しかしそれでも接続がうまくいかないこともあり、大体2,3分待って始まらなければ「実行部分」を停止→再度実行した方が良いことが多いです。

*   なお時々多行のエラーが出力されることがありますが、その際は申し訳ございませんがランタイムの接続を解除して「環境構築」からやり直した方がいいかと思われます。

*   上記外でもスクレイピングの途中でエラーが起きた場合は自動的にシステムが再実行されるようにコードを組んでいるつもりですが、確認が至ってなくバグが残っている箇所もあるかもしれません。その点はご容赦ください。

### **引数説明**

- **disease**：集める感染症データの指定に使います。WAHISデータベースのFilter機能に載っているもの（英語）であり、かつ一意に定まる限り（感染症名の全てを入力しなくても）対応できます。リスト[ ]の中に対象の疾患を'  'または"  "で囲んで文字列（str）として入力してください。複数指定可。初期値はないので必ず何かしら指定をしてください。

- **Event_status**：感染状況の指定に使います。選択肢としては"On-going", "Resolved", "Stable"の3つが取れます。リストで入力してください。複数指定可。初期値はNoneで、感染状況に関係なくスクレイピングされます。

- **Country**：感染報告のある国の指定に使います。WAHISデータベースのFilter機能に載っているもの（英語）であり、かつ一意に定まる限り対応できます。リスト[ ]の中に対象の国を'  'または"  "で囲んで文字列（str）として入力してください。複数指定可。初期値はNoneです。

- **World_region**：感染報告のある地域の指定に使います。選択肢としては"Africa", "Americas", "Asia, "Europe", "Oceania", "Other"の6つが取れます。リストで入力してください。複数指定可。初期値はNoneです。

- **SubType**：病原体の型の指定に使います。WAHISデータベースのFilter機能に載っているもの（英語）であり、かつ一意に定まる限り対応できます。リスト[ ]の中に対象の型を'  'または"  "で囲んで文字列（str）として入力してください。複数指定可。初期値はNoneです。

- **Summary, Detail**：既に収集データがあり、残りのスクレイピングを進めたい、または前回収集時から新規に追加されたデータを加えたい場合に使います。左端の📁カラム中に該当データをアップロードしてもらった後、引数で**"/content/~~"** （e.g. "/content/Summary.csv"）のようにパスを入れてください。初期値はNoneです。

- **previous_date**：前回収集時から新規に追加されたデータを既存ファイルに加える場合、その「前回収集」の日時を"YYYY/MM/DD"で記載してください。初期値はNoneです。

- **from_date, to_date**：集める感染症データの期間指定に使います。"YYYY/MM/DD"で記載してください。個々のリスト及びレポートにおいて、指定期間内に少しでも重なっていればスクレイピング対象となります。初期値はNoneです。

- **nowPage**：スクレイピング第一段階で実行を止めた際に使います。CorrectList.nowPageから現在取得したIDのページ数が分かるため、それをそのまま数値（int）で入れてください。再度実行した際にそこからスクレイピングを再開します。初期値はNoneです。

- **nowList, nowReport**：スクレイピング第二段階で実行を止めた際に使います。CorrectSummary.all_thread_data_S, CorrectDetail.all_thread_data_Dで現在の進捗状況が分かるため、そこで得られた辞書（dict）をそれぞれnowList, nowReportに入れてください。再度実行した際にそこからスクレイピングを再開します。初期値はどちらも{}です。

### **出力データについて**

#### **Summary_sorted.csv**

データ作成の関係上、1列目及び1行目は数値でのindexです。

2行目に関してSummary, Control Measure, Quantitativeとありますが、データベース先に基づいて情報の種類を記載しています。

4行目に詳細なカラム名があります。

- **eventId**：該当報告（リスト）のID

- **country.areaId, country.name**：該当報告の発生国のID及び名前

- **startedOn, lastOccurrence, endOn**：該当報告の初発日、前回発生日、終息日

- **eventStatus.id, eventStatus.translation**：該当報告の発生状況

- **isAquatic**：該当報告が魚類でのものかどうか

- **disease.diseaseId, disease.name**：疾患ID、疾患名

- **subType.diseaseId, subType.name**：原因病原体の型及びID

- **reason.id, reason.translation**：原因病原体についての情報及びそのID

- **clinicalSigns**：臨床兆候の有無

- **sources.sourceId, sources.name**：発生原因及びそのID

- **evenetComment**：その他コメント

- **methods.id, methods.translation**：診断手法のID及び名前

- **laboratoryTest.testName, confirmOn, laboratoryTests.result**：診断に際して実験室で行われた検査、実施日時、結果

- **Controls Measure**：3行目に家畜または野外への適用の分類、4行目に実施された防疫対策の項目。適用であれば"Applied", それ以外は空欄もしくは"-"

- **id, Name（Quantitative）**：動物頭数の単位（基本的に"体"であり、"群"などはないかと思いますが…）

- **Quantitative**：3行目に動物種、4行目に分類（Total- / New-）+ （Susceptible, Cases, Deaths, Killed, Slaughtered, Vaccinated）


#### **Detail_sorted.csv**

データ作成の関係上、1列目及び1行目は数値でのindexです。

2行目に関してSummary, Control Measure, Quantitativeとありますが、データベース先に基づいて情報の種類を記載しています。

4行目に詳細なカラム名があります。

- **eventId**：該当報告（リスト）のID

- **outbreakId**：該当報告（レポート）のID

- **areaId, latitude, longtitude**：該当報告の発生地域のID、座標

- **epiUnitType_id, epiUnitType_translation**：発生地域の地域単位（村や農村など）及びID

- **startDate, endDate**：該当報告の初発日、終息日

- **diagnosticMethods.id, diagnosticMethods.translation**：診断手法のID及び名前

- **adminDivision.?.id, adminDivision.?.name**：行政管理区分のID及び名前

- **Controls Measure**：4行目に実施された防疫対策の項目。適用であれば"Applied", それ以外は空欄もしくは"-"

- **id, Name（Quantitative）**：動物頭数の単位及びID（基本的に"体"であり、"群"などはないかと思いますが…）

- **Quantitative**：3行目に動物種、4行目に分類（Total- / New-） + （Susceptible, Cases, Deaths, Killed, Slaughtered, Vaccinated）

# ゴミ箱

## マルチスレッド化に伴い廃棄

### APIからの情報取得 Ver.2.5

参考 \\
https://youtu.be/et-cDFbVkQw?si=rusBs1kgbXV9V1MK \\
https://qiita.com/tchnkmr/items/b05f321fa315bbce4f77

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG, format="%(processName)s: %(message)s")


class CorrectDetail():
    '''
    class_variables = {
        'nowReport': None,
        'DQ_param': None,
    }

    @classmethod
    def update_class_variable(cls, variable_name, value):
        update_class_variable(cls, variable_name, value)
    '''

    @classmethod
    def update_classvalues(cls, nowReport=None):
        cls.nowReport = nowReport

    def __init__(self, df2, from_date, DQ_param):
        self.DQ_param = DQ_param
        self.df2 = df2
        self.from_date = from_date
        self.SR = SendRequest()

    def correct_detaildata(self, eventId, extracted_previewdata, list_number):
        if CorrectDetail.nowReport:
            number = CorrectDetail.nowReport
        else:
            CorrectDetail.nowReport = 0
            number = 0

        if not self.df2.empty:
            where_eve = self.df2.columns[self.df2.iloc[0] == str(eventId)]
        else:
            where_eve = pd.Index([])

        with ThreadPoolExecutor(max_workers=2) as executor:
            while number + 1 <= len(extracted_previewdata):
                #logging.debug(f"Thread {executor.name} is running")
                future = executor.submit(self.process_single_detail, eventId, extracted_previewdata, number, where_eve, list_number)
                future.result()  # 処理完了を待つ
                number += 1
                CorrectDetail.nowReport += 1

        print("Get all report in one list!")
        CorrectDetail.nowReport = None

    def process_single_detail(self, eventId, extracted_previewdata, number, where_eve, list_number):
        response = self.SR.send_request_with_cookie(self.DQ_param, 'GET', [eventId, extracted_previewdata[number]])
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            return

        response_data = response.json()
        ref_col, check = self.check_date_and_ref_D(response_data)
        time.sleep(3)

        if check:
            processor1 = Qualitative_DataFormat(response_data, (list_number + number))
            Integrated_Info = processor1.integrate_information(eventId)

            with ProcessPoolExecutor(max_workers=2) as process_executor:  # マルチプロセス化
                #future1 = process_executor.submit(Qualitative_DataFormat, response_data, (list_number + number))
                future2 = process_executor.submit(ControlDataFormat, response_data, self.df2, (list_number + number))
                future3 = process_executor.submit(QuantitativeDataFormat, response_data, self.df2, (list_number + number))

            #Integrated_Info = future1.result().integrate_information(eventId)
            Control_content, df2_first = future2.result().get_control_content()
            Quant_data, df2_latter = future3.result().update_ws_quant_data()

            self.df2 = pd.concat([df2_first, df2_latter], axis=0).reset_index(drop=True)
            Data = pd.concat([Integrated_Info, Control_content, Quant_data], axis=0).astype(str).reset_index(drop=True)

            if ref_col is not None:
                self.df2[ref_col] = Data['value'] if (list_number + number) == 0 else Data
            else:
                if not where_eve.empty:
                    new_row_col = int(where_eve[-1]) + 1
                    self.df2 = pd.concat([self.df2.iloc[:, :new_row_col], Data, self.df2.iloc[:, new_row_col:]], axis=1, ignore_index=True).reset_index(drop=True)
                else:
                    self.df2 = pd.concat([self.df2, Data], axis=1, ignore_index=True).reset_index(drop=True)

        #print(self.df2)

    def check_date_and_ref_D(self, item):
        outbreak = pd.json_normalize(item['outbreak']).iloc[0]
        outbreakId = outbreak['outbreakId']
        Enddate = pd.to_datetime(outbreak['endDate']).date().strftime('%Y-%m-%d') if outbreak['endDate'] else '-'

        if not self.df2.empty:
            where_ref = self.df2.columns[self.df2.iloc[1] == str(outbreakId)]
        else:
            where_ref = pd.Index([])

        if self.from_date is not None:
            date_list = [Enddate, self.from_date]
            check = self.CD.narrow_down(date_list)
        else:
            check = False

        return where_ref[0] if not where_ref.empty else None, (not check)


In [ ]:
class CorrectSummary(CorrectDetail):
    @classmethod
    def update_classvalues(cls, nowList=None):
        cls.nowList = nowList

    def __init__(self, df1, df2, from_date, PQ_param, DQ_param):
        self.PQ_param = PQ_param
        self.df1 = df1
        self.from_date = from_date
        self.SR = SendRequest()
        super().__init__(df2, from_date, DQ_param)

    def correct_previewdata(self, extracted_listdata):
        if CorrectSummary.nowList:
            number = CorrectSummary.nowList
        else:
            CorrectSummary.nowList = 0
            number = 0

        with ThreadPoolExecutor(max_workers=2) as executor:
            while number + 1 <= len(extracted_listdata):
                future = executor.submit(self.process_single_preview, extracted_listdata, number)
                future.result()  # 完了を待つ
                number += 1
                CorrectSummary.nowList += 1

        print("Get all report in one list!")

    def process_single_preview(self, extracted_listdata, number):
        response = self.SR.send_request_with_cookie(self.PQ_param, 'GET', [extracted_listdata[number]['eventId']])
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            return

        response_data = response.json()
        ref_col, check = self.check_date_and_ref_S(response_data)

        if check:
            processor1 = Qualitative_DataFormat(response_data, number)
            Integrated_Info = processor1.integrate_information()

            with ProcessPoolExecutor(max_workers=2) as process_executor:
                #future1 = process_executor.submit(Qualitative_DataFormat, response_data, number)
                future2 = process_executor.submit(ControlDataFormat, response_data, self.df1, number)
                future3 = process_executor.submit(QuantitativeDataFormat, response_data, self.df1, number)

            #Integrated_Info = future1.result().integrate_information()
            Control_content, df1_first = future2.result().get_control_content()
            Quant_data, df1_latter = future3.result().update_ws_quant_data()

            self.df1 = pd.concat([df1_first, df1_latter], axis=0).reset_index(drop=True)
            Data = pd.concat([Integrated_Info, Control_content, Quant_data], axis=0).astype(str).reset_index(drop=True)

            if ref_col is not None:
                self.df1[ref_col] = Data['value'] if number == 0 else Data
            else:
                self.df1 = pd.concat([self.df1, Data], axis=1, ignore_index=True).reset_index(drop=True)

        outbreak_df = pd.json_normalize(response_data['outbreaks'])
        if not outbreak_df.empty:
            outbreak_ids = outbreak_df['outbreakId'].tolist()
            self.correct_detaildata(extracted_listdata[number]['eventId'], outbreak_ids, number)
        self.df1.to_csv('Summary.csv', index=False)
        self.df2.to_csv('Detail.csv', index=False)


    def check_date_and_ref_S(self, item):
        events = pd.json_normalize(item['event']).iloc[0]
        eventId = events['eventId']
        Enddate = pd.to_datetime(events['endedOn']).date().strftime('%Y-%m-%d') if events['endedOn'] else '-'

        if not self.df1.empty:
            where_ref = self.df1.columns[self.df1.iloc[0] == str(eventId)]
        else:
            where_ref = pd.Index([])

        if self.from_date is not None:
            date_list = [Enddate, self.from_date]
            check = self.CD.narrow_down(date_list)
        else:
            check = False

        return where_ref[0] if not where_ref.empty else None, (not check)


In [ ]:
class CorrectList(CorrectSummary):
    @classmethod
    def update_classvalues(cls, nowPage=None, finished=False, extracted_listdata=[]):
        cls.nowPage = nowPage
        cls.finished = finished
        cls.extracted_listdata = extracted_listdata

    def __init__(self, df1, df2, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param):
        self.FQ_param = FQ_param
        self.to_date = to_date
        self.CD = CompareDate()
        self.SR = SendRequest()
        self.pre_date = previous_date
        self.df1 = df1
        self.df2 = df2
        super().__init__(df1, df2, from_date, PQ_param, DQ_param)

    def scrape(self):
        if not CorrectList.finished:
            self.correct_listdata()
        self.correct_previewdata(CorrectList.extracted_listdata)
        self.final_sort(1, [0], [False])
        self.final_sort(2, [0, 1], [False, False])


    def correct_listdata(self):
        # リクエストを送信し、結果を表示
        if CorrectList.nowPage:
            self.FQ_param["body"]["pageNumber"] = CorrectList.nowPage
        else:
            CorrectList.nowPage = 0

        while True:
            print(CorrectList.nowPage)

            response = self.SR.send_request_with_cookie(self.FQ_param, 'POST')
            if response.status_code != 200:
                print(f"Error: Received status code {response.status_code}")
                break

            response_data = response.json()

            if not response_data.get('list'):
                print("No more data available.")
                CorrectList.finished = True
                break

            # 必要な情報を抽出してリストに整理
            for item in response_data['list']:
                check = self.check_date(item)

                if check:
                    CorrectList.extracted_listdata.append({
                        'eventId': item['eventId']
                    })

            # 次のページへ
            self.FQ_param["body"]["pageNumber"] += 1
            CorrectList.nowPage += 1

    def check_date(self, item):
        Update_date = pd.to_datetime(item['submissionDate']).date() if item['submissionDate'] is not None else '-'
        Start_date = pd.to_datetime(item['eventStartDate']).date() if item['eventStartDate'] is not None else '-'

        if self.pre_date is not None:
            date_list = [Update_date, self.pre_date]
            checkA = self.CD.narrow_down(date_list)
        else:
            checkA = False

        if self.to_date is not None:
            date_list2 = [self.to_date, Start_date]
            checkB = self.CD.narrow_down(date_list2)
        else:
            checkB = False

        return not (checkA or checkB)

    def final_sort(self, number, columns, ascending_list):
        if number == 1:
            df = self.df1
            name = "Summary"
        elif number == 2:
            df = self.df2
            name = "Detail"

        df = df.T.reset_index()
        df_subset = df.iloc[3:, :]
        df_subset[columns] = df_subset[columns].astype(int)
        df_sorted_subset = df_subset.sort_values(by=columns, ascending=ascending_list)
        df.iloc[3:, :] = df_sorted_subset
        df[columns] = df[columns].astype(str)
        df.drop(columns=['index'], axis=1).reset_index().to_csv(f'{name}_sorted.csv', index=False)


### 統合 Ver.2

参考 \\
https://qiita.com/puchi2121/items/3983a75c8c0a6e958fbd

In [ ]:
class AutoReportScraper(CorrectList):
    def __init__(self, df1, df2, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param):
        self.df1 = df1
        self.df2 = df2
        super().__init__(df1, df2, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param)

    def scrape_reports_with_retry(self):
        n = 0
        prev_list = []
        same = 0
        while True:
          try:
              if n == 0:
                self.scrape()
              else:
                print("Restarted!")
                self.scrape()
          except Exception as e:
              n += 1
              print(f"An error happened: {e}")
              self.df1.to_csv('Summary.csv', index=False)
              self.df2.to_csv('Detail.csv', index=False)
              #print(self.df1)
              #print(self.df2)
              etype, value, tb = sys.exc_info()
              print(traceback.format_exception(etype, value, tb))
              # エラーが発生した場合、現在の状態を取得して再実行する
              now_list = [CorrectList.nowPage, CorrectSummary.nowList, CorrectDetail.nowReport]
              print(now_list)
              self._get_current_state(CorrectList.nowPage, CorrectSummary.nowList, CorrectDetail.nowReport, CorrectList.finished, CorrectList.extracted_listdata)
              if (prev_list != []) and (set(prev_list) == set(now_list)):
                same += 1
                if same >= 5:
                  break
              if "429" in str(e):
                print("waiting 10 seconds ...")
                time.sleep(10)
                same -= 1
              prev_list = now_list
              time.sleep(2)
          else:
              self.df1.to_csv('Summary.csv', index=False)
              #print(self.df1)
              self.df2.to_csv('Detail.csv', index=False)
              #print(self.df2)
              break


    def _get_current_state(self, Page, List, Report, Finished, eventId_list):
        CorrectList.nowPage = Page
        CorrectSummary.nowList = List
        CorrectDetail.nowReport = Report
        CorrectList.finished = Finished
        CorrectList.extracted_listdata = eventId_list

In [ ]:
class ScrapingWAHIS():
    def __init__(self, disease, df1 = None, df2 = None, previous_date = None, from_date = None, to_date = None,
                 nowPage = None, nowList = None, nowReport = None):
        self.disease = disease
        self.df1 = df1 if df1 is not None else pd.DataFrame()
        self.df2 = df2 if df2 is not None else pd.DataFrame()
        self.pre_date = previous_date
        self.fd = from_date
        self.td = to_date

        CorrectList.update_classvalues(nowPage)
        CorrectSummary.update_classvalues(nowList)
        CorrectDetail.update_classvalues(nowReport)


    #@retry(retry=retry_if_exception_type((TimeoutError, APIError, HTTPError)))
    def run_scraping(self):
        n = 0
        while True:
          try:
              if n == 0:
                  browser = HeadlessBrowser()
                  FQ_param, PQ_param, DQ_param = browser.visit_url(self.disease)
                  break
              else:
                  print("Restarted before scraping!")
                  browser = HeadlessBrowser()
                  FQ_param, PQ_param, DQ_param = browser.visit_url(self.disease)
                  break
          except Exception as e:
              n += 1
              print(f"An error happened in preparing: {e}")
              time.sleep(2)

        print("start")

        ARS = AutoReportScraper(self.df1, self.df2, self.pre_date, self.fd, self.td, FQ_param, PQ_param, DQ_param)
        ARS.scrape_reports_with_retry()

## gspread→pandasに伴い廃棄

In [ ]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

###  APIからの情報取得

In [ ]:
class CorrectDetail():
    '''
    class_variables = {
        'nowReport': None,
        'DQ_param': None,
    }

    @classmethod
    def update_class_variable(cls, variable_name, value):
        update_class_variable(cls, variable_name, value)
    '''

    @classmethod
    def update_classvalues(cls, nowReport = None):
        cls.nowReport = nowReport

    def __init__(self, ws2, ss, from_date, DQ_param):
        self.DQ_param = DQ_param
        self.ws2 = ws2
        self.ss = ss
        self.from_date = from_date
        self.SR = SendRequest()

    def correct_detaildata(self, eventId, extracted_previewdata, list_number):
        if CorrectDetail.nowReport:
            number = CorrectDetail.nowReport
            #record = CorrectDetail.nowReport
        else:
            CorrectDetail.nowReport = 0
            number = 0
            #record = 0

        where_eve = self.ws2.findall(str(eventId), in_column=1)

        while number+1 <= len(extracted_previewdata):
            response = self.SR.send_request_with_cookie(self.DQ_param, 'GET', [eventId, extracted_previewdata[number]])
            if response.status_code != 200:
                  print(f"Error: Received status code {response.status_code}")
                  break

            response_data = response.json()
            ref_index, check = self.check_date_and_ref_D(response_data)
            time.sleep(3)

            if check:
                processor1 = Qualitative_DataFormat(response_data, (list_number+number))
                Integrated_Info = processor1.integrate_information(eventId)

                processor2 = ControlDataFormat(response_data, self.ws2, (list_number+number))
                Control_content = processor2.get_control_content()

                processor3 = QuantitativeDataFormat(response_data, self.ws2, (list_number+number))
                Quant_data = processor3.update_ws_quant_data()

                Data = pd.concat([Integrated_Info, Control_content, Quant_data], axis=0)
                Data = Data.transpose().astype(str).values.tolist()

                if ref_index != None:
                    column_count = len(Data[0])
                    cell_range = f'A{ref_index.row}:{get_column_letter(column_count)}{ref_index.row}'
                    if (list_number+number) == 0:
                        self.ws2.update(range_name = cell_range, values = [Data[3]], value_input_option="USER_ENTERED")
                    else:
                        self.ws2.update(range_name = cell_range, values = Data, value_input_option="USER_ENTERED")
                else:
                    if where_eve != []:
                        self.ws2.insert_rows(Data, where_eve[-1].row+1, value_input_option="USER_ENTERED")
                    else:
                        self.ss.values_append("Detail", {"valueInputOption": "USER_ENTERED"}, {"values": Data})

            number += 1
            CorrectDetail.nowReport += 1
            time.sleep(3)

        print("Get all report in one list!")
        CorrectDetail.nowReport = None

    def check_date_and_ref_D(self, item):
        outbreak = pd.json_normalize(item['outbreak']).iloc[0]
        outbreakId = outbreak['outbreakId']
        Enddate = pd.to_datetime(outbreak['endDate']).date().strftime('%Y-%m-%d') if outbreak['endDate'] else '-'

        where_ref = self.ws2.find(str(outbreakId), in_column=2)

        if self.from_date != None:
              date_list = [Enddate, self.from_date]
              check = self.CD.narrow_down(date_list)
        else:
              check = False

        return  where_ref, (not check)

In [ ]:
class CorrectSummary(CorrectDetail):
    @classmethod
    def update_classvalues(cls, nowList = None):
        cls.nowList = nowList

    def __init__(self, ws1, ws2, ss, from_date, PQ_param, DQ_param):
        self.PQ_param = PQ_param
        self.ws1= ws1
        self.ss = ss
        self.from_date = from_date
        self.SR = SendRequest()
        super().__init__(ws2, ss, from_date, DQ_param)

    def correct_previewdata(self, extracted_listdata):
        if CorrectSummary.nowList:
            number = CorrectSummary.nowList
            #record = CorrectSummary.nowList
        else:
            CorrectSummary.nowList = 0
            number = 0
            #record = 0

        while number+1 <= len(extracted_listdata):
            response = self.SR.send_request_with_cookie(self.PQ_param, 'GET', [extracted_listdata[number]['eventId']])
            if response.status_code != 200:
                print(f"Error: Received status code {response.status_code}")
                break
            else:
               print("Gotcha!")

            response_data = response.json()
            ref_index, check = self.check_date_and_ref_S(response_data)

            if check:
                processor1 = Qualitative_DataFormat(response_data, number)
                Integrated_Info = processor1.integrate_information()

                processor2 = ControlDataFormat(response_data, self.ws1, number)
                Control_content = processor2.get_control_content()

                processor3 = QuantitativeDataFormat(response_data, self.ws1, number)
                Quant_data = processor3.update_ws_quant_data()

                Data = pd.concat([Integrated_Info, Control_content, Quant_data], axis=0)
                Data = Data.transpose().astype(str).values.tolist()

                if ref_index != None:
                    column_count = len(Data[0])
                    cell_range = f'A{ref_index.row}:{get_column_letter(column_count)}{ref_index.row}'
                    if number == 0:
                        self.ws1.update(range_name = cell_range, values = [Data[3]], value_input_option="USER_ENTERED")
                    else:
                        self.ws1.update(range_name = cell_range, values = Data, value_input_option="USER_ENTERED")
                else:
                    self.ss.values_append("Summary", {"valueInputOption": "USER_ENTERED"}, {"values": Data})
                    print(Data)


            outbreak_df = pd.json_normalize(response_data['outbreaks'])
            outbreak_ids = outbreak_df['outbreakId'].tolist()
            self.correct_detaildata(extracted_listdata[number]['eventId'], outbreak_ids, number)

            number += 1
            CorrectSummary.nowList += 1


    def check_date_and_ref_S(self, item):
        events = pd.json_normalize(item['event']).iloc[0]
        eventId = events['eventId']
        Enddate = pd.to_datetime(events['endedOn']).date().strftime('%Y-%m-%d') if events['endedOn'] else '-'

        where_ref = self.ws1.find(str(eventId), in_column=1)

        if self.from_date != None:
              date_list = [Enddate, self.from_date]
              check = self.CD.narrow_down(date_list)
        else:
              check = False

        return  where_ref, (not check)

In [ ]:
class CorrectList(CorrectSummary):
    @classmethod
    def update_classvalues(cls, nowPage = None, finished = False, extracted_listdata = []):
        cls.nowPage = nowPage
        cls.finished = finished
        cls.extracted_listdata = extracted_listdata

    def __init__(self, ws1, ws2, ss, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param):
        self.FQ_param = FQ_param
        self.to_date = to_date
        self.CD = CompareDate()
        self.SR = SendRequest()
        self.pre_date = previous_date
        super().__init__(ws1, ws2, ss, from_date, PQ_param, DQ_param)

    def scrape(self):
        if not CorrectList.finished:
            self.correct_listdata()
        self.correct_previewdata(CorrectList.extracted_listdata)

    def correct_listdata(self):
        # リクエストを送信し、結果を表示
        if CorrectList.nowPage:
            self.FQ_param["body"]["pageNumber"] = CorrectList.nowPage
            #record = CorrectList.nowPage
        else:
            CorrectList.nowPage = 0
            #record = 0


        while True:
            print(CorrectList.nowPage)

            response = self.SR.send_request_with_cookie(self.FQ_param, 'POST')
            if response.status_code != 200:
                  print(f"Error: Received status code {response.status_code}")
                  break

            response_data = response.json()

            if not response_data.get('list'):
                print("No more data available.")
                CorrectList.finished = True
                break

            # 必要な情報を抽出してリストに整理
            for item in response_data['list']:
                check = self.check_date(item)

                if check:
                    CorrectList.extracted_listdata.append({
                        #'reportId': item['reportId'],
                        'eventId': item['eventId'],
                        #'country': item['country'],
                        #'disease': item['disease'],
                        #'subType': item['subType'],
                        #'eventStartDate': item['eventStartDate'],
                        #'eventStatus': item['eventStatus'],
                        #'reason': item['reason'],
                        #'reportType': item['reportType'],
                        #'reportStatus': item['reportStatus'],
                        #'submissionDate': item['submissionDate'],
                        #'reportNumber': item['reportNumber'],
                        #'isAquatic': item['isAquatic']
                    })

            # 次のページへ
            self.FQ_param["body"]["pageNumber"] += 1
            CorrectList.nowPage += 1

        #return CorrectList.extracted_listdata

    def check_date(self, item):
        Update_date = pd.to_datetime(item['submissionDate']).date() if item['submissionDate'] is not None else '-' # Not euqal to EndDate even if 'eventStatus' is 'Resolved'
        Start_date = pd.to_datetime(item['eventStartDate']).date() if item['eventStartDate'] is not None else '-'

        if self.pre_date != None:
              date_list = [Update_date, self.pre_date]
              checkA = self.CD.narrow_down(date_list)
        else:
              checkA = False

        if self.to_date != None:
              date_list2 = [self.to_date, Start_date]
              checkB = self.CD.narrow_down(date_list2)
        else:
              checkB = False

        return  not (checkA or checkB)


### データ整理クラス関数

In [ ]:
class Qualitative_DataFormat:
    def __init__(self, response_data, number):
        self.response_data = response_data
        self.number = number

    def create_common_info_S(self):
        events = self.response_data['event']
        #reports = pd.json_normalize(self.response_data['report']).iloc[0]
        sources = pd.json_normalize(self.response_data['sources'])
        methods = pd.json_normalize(self.response_data['methods'])
        laboratoryTests = pd.json_normalize(self.response_data['laboratoryTests'])
        #print(events)

        common_info = {
            'eventId': events['eventId'],
            'country.areaId': events['country']['areaId'],
            'country.name': events['country']['name'],
            'country.isoCode': events['country']['isoCode'],
            'startedOn': pd.to_datetime(events['startedOn']).date() if events['startedOn'] is not None else '-',
            'lastOccurrence': pd.to_datetime(events['lastOccurrence']).date() if events['lastOccurrence'] is not None else '-',
            'endedOn': pd.to_datetime(events['endedOn']).date() if events['endedOn'] is not None else '-',
            'eventStatus.id': events['eventStatus']['id'],
            'eventStatus.translation': events['eventStatus']['translation'],
            'isAquatic': events['isAquatic'],
            'disease.diseaseId': events['disease']['diseaseId'],
            'disease.name': events['disease']['name'],
            'subType.diseaseId': events['subType']['disease']['diseaseId'] if events['subType']['disease'] is not None else 'Pending',
            'subType.name': events['subType']['disease']['name'] if events['subType']['disease'] is not None else 'Pending',
            'reason.id': events['reason']['id'],
            'reason.translation': events['reason']['translation'],
            'clinicalSigns': events['clinicalSigns'],
            'sources.sourceId': sources['source.sourceId'].tolist(),
            'sources.name': sources['source.name'].tolist(),
            'eventComment': events['eventComment']['translation'] if events['eventComment'] is not None else 'No epidemiological comment',
            'methods.id': methods['nature.id'].tolist(),
            'methods.translation': methods['nature.translation'].tolist(),
            'laboratoryTests.testName': laboratoryTests['testName'].tolist(),
            'confirmOn': pd.to_datetime(events['confirmOn']).date() if events['confirmOn'] is not None else '-',
            'laboratoryTests.result': laboratoryTests['result'].tolist(),
            # 他に必要な情報を追加する
        }

        return common_info

    def create_common_info_D(self, eventId):
        outbreak = pd.json_normalize(self.response_data['outbreak']).iloc[0]
        adminDivisions = self.response_data['adminDivisions']
        diagnosticMethods = pd.json_normalize(self.response_data['diagnosticMethods'])

        common_info = {
            'eventId': eventId,
            'outbreakId': outbreak['outbreakId'],
            'areaId': outbreak['areaId'],
            'latitude': outbreak['latitude'],
            'longtitude': outbreak['longitude'],
            'epiUnitType_id': outbreak['epiUnitType.id'],
            'epiUnitType_translation': outbreak['epiUnitType.translation'],
            'startDate': pd.to_datetime(outbreak['startDate']).date() if outbreak['startDate'] else '-',
            'endDate': pd.to_datetime(outbreak['endDate']).date() if outbreak['endDate'] else '-',
            'diagnosticMethods.id': diagnosticMethods['nature.id'].tolist() if self.response_data['diagnosticMethods'] != [] else '-',
            'diagnosticMethods.translation': diagnosticMethods['nature.translation'].tolist() if self.response_data['diagnosticMethods'] != [] else '-',
            # 他に必要な情報を追加する
        }

        for i in range(3):
            common_info[f'adminDivisions.{i+1}.id'] = '-'
            common_info[f'adminDivisions.{i+1}.name'] = '-'
        for item in adminDivisions:
            L = item['adminLevel']
            common_info[f'adminDivisions.{L}.id'] = item['areaId']
            common_info[f'adminDivisions.{L}.name'] = item['name']

        return common_info

    def integrate_information(self, eventId = None):
        if 'event' in list(self.response_data.keys()):
            info = self.create_common_info_S()
        elif 'outbreak' in list(self.response_data.keys()):
            info = self.create_common_info_D(eventId)

        df = pd.DataFrame()
        df['Item'] = ['Summary']*len(info)
        df['species'] = ['']*len(info)
        df['category'] = list(info.keys())
        df['value'] = list(info.values())

        if self.number == 0:
          Integrated_Info = df
        else:
          Integrated_Info = df.filter(items = ['value'])

        return Integrated_Info


In [ ]:
class ControlDataFormat:
    def __init__(self, resoponse_data, ws, number):
        self.resoponse_data = resoponse_data
        self.ws = ws
        self.number = number

    def extract_measures_info_S(self, measures_data):
        measures_name = []
        domestic_value = []
        wild_value = []

        for index, measure in measures_data.iterrows():
            measures_name.append(measure['measure.name'])

            # デフォルトの値を設定
            domestic_status = '-'
            wild_status = '-'

            category_original = measure['measure.category.original'].lower()
            if 'domestic' in category_original:
                domestic_status = measure['status.translation']
            elif 'wild' in category_original:
                wild_status = measure['status.translation']

            domestic_value.append(domestic_status)
            wild_value.append(wild_status)

        df = pd.DataFrame()
        df['Item'] = ['Control Measure']*len(measures_name)*2
        df['species'] = [item for item in ['Domestic', 'Wild'] for _ in range(len(measures_name))]
        df['category'] = measures_name*2
        df['value'] = domestic_value + wild_value
        report_cont = list(dict.fromkeys(measures_name))

        return df, report_cont

    def extract_measures_info_D(self):
        measure_name = []
        #additional_measure = []
        measure_apply = []
        measure = self.resoponse_data['controlMeasures']
        add_measures = self.resoponse_data['additionalMeasures']

        for item in measure:
            measure_name.append(item['measure']['name'])

            # デフォルトの値を設定
            default_status = '-'
            if item['status']['translation'] == 'Applied':
                measure_apply.append('Applied')
            else:
                measure_apply.append(default_status)

        for item in add_measures:
            measure_name.append(item['name'])
            measure_apply.append('Applied')

        df = pd.DataFrame()
        df['Item'] = ['Control Measure']*len(measure_apply)
        df['species'] = ['' for _ in range(len(measure_apply))]
        df['category'] = measure_name# + additional_measure
        df['value'] = measure_apply
        report_cont = list(dict.fromkeys(df['category']))

        return df, report_cont

    def get_cont_unique(self):
        cells = self.ws.findall('Control Measure')
        cont_unique = []
        for a in range(len(cells)):
            spread_cont_content = self.ws.cell(cells[a].row+2, cells[a].col).value
            cont_unique.append(spread_cont_content)
        return list(dict.fromkeys(cont_unique))

    def update_ws_cont_data(self, cont_unique, report_cont, control_df):

        order, order_name, record = self.get_order_and_record(cont_unique, report_cont, control_df)

        Cont_df = self.get_after_cont_content(order, order_name, record, report_cont, cont_unique, control_df)

        Cont_data = self.add_missing_species(Cont_df, cont_unique, order)

        return Cont_data

    def get_order_and_record(self, cont_unique, report_cont, control_df):
        order = []
        order_name = []
        record = []
        insert_ver = True if cont_unique == [] else False
        for mea in report_cont:
            try:
                order.append(cont_unique.index(mea))
                order_name.append(mea)
            except ValueError:
                record.append(mea)
                self.insert_new_cols(mea, control_df, insert_ver)
        return order, order_name, record

    def insert_new_cols(self, cont_category, df, insert_ver):
        if insert_ver:
            cells = self.ws.findall('Summary')
            Insert_col = df[df['category'] == cont_category].drop('value', axis=1).astype(str).values.tolist()
            self.ws.insert_cols(Insert_col, col=cells[-1].col+1)
        else:
            cells = self.ws.findall('Control Measure')
            Insert_col = df[df['category'] == cont_category].drop('value', axis=1).astype(str)
            print("here1")
            print(self.resoponse_data['event'])
            if self.resoponse_data['event'] != []:
                Insert_col_D = Insert_col[Insert_col['species'] == 'Domestic'].values.tolist()
                self.ws.insert_cols(Insert_col_D, col=cells[len(cells)//2 - 1].col+1)
                Insert_col_W = Insert_col[Insert_col['species'] == 'Wild'].values.tolist()
                self.ws.insert_cols(Insert_col_W, col=cells[-1].col+1+int(len(Insert_col)/2))
            else:
                Insert_col = Insert_col.values.tolist()
                self.ws.insert_cols(Insert_col, col=cells[-1].col+1)


    def get_after_cont_content(self, order, order_name, record, report_cont, cont_unique, control_df):
        order_sorted = sorted(order)
        after_Cont_content = pd.DataFrame()
        if order != order_sorted:
            for name in order_name:
                df_to_append = control_df[control_df['category'] == name]
                after_Cont_content = pd.concat([after_Cont_content, df_to_append], ignore_index=True)
        else:
            after_Cont_content = control_df

        if len(record) != 0:
            for name2 in record:
              df_to_append2 = control_df[control_df['category'] == name2]
              after_Cont_content = after_Cont_content[after_Cont_content['category'] != name2]
              after_Cont_content = pd.concat([after_Cont_content, df_to_append2], ignore_index=True)

        return after_Cont_content

    def add_missing_species(self, Cont_df, cont_unique, order):
        if len(cont_unique) != len(order):
            Cont_list = Cont_df['value'].astype(str).tolist()
            diff_set = set([h for h in range(len(cont_unique))]) - set(order)
            missing = sorted([diff for diff in diff_set])
            print("here2")
            print(self.resoponse_data.keys())
            if (Cont_list ==  []) and ('event' in self.resoponse_data.keys()):
                Cont_list = np.full(len(missing)*2, '-', dtype=object)
            elif (Cont_list ==  []):
                Cont_list = np.full(len(missing), '-', dtype=object)
            else:
                for c in range(len(missing)):
                    Cont_list = np.insert(np.reshape(Cont_list, (2, -1)), missing[c], np.array(['-' for _ in range(2)], dtype=object), axis=1).ravel()
            return pd.DataFrame(Cont_list.tolist(), columns=['value'])
        else:
            return Cont_df.filter(items=['value'])

    def get_control_content(self):
        if 'measures' in list(self.resoponse_data.keys()):
            measures_data = pd.json_normalize(self.resoponse_data['measures'])
            measures_df, report_cont = self.extract_measures_info_S(measures_data)
        else:
            measures_df, report_cont = self.extract_measures_info_D()
        cont_unique = self.get_cont_unique()

        if self.number == 0:
            Control_content = measures_df
        elif cont_unique != report_cont:
            Control_content = self.update_ws_cont_data(cont_unique, report_cont, measures_df)
        else:
            Control_content = measures_df.filter(items = ['value'])

        return Control_content


In [ ]:
class QuantitativeDataFormat:
    def __init__(self, response_data, ws, number):
        self.response_data = response_data
        self.ws = ws
        self.number = number

    def preprocessing(self, p_type):
        if p_type == 'Summary':
            quantitative_data = self.response_data['quantitativeData']
            news_data = quantitative_data.get('news', [])
            totals_data = quantitative_data.get('totals', [])
            unit_data = quantitative_data.get('unit', [])
        elif p_type == 'Detail':
            unit_data = self.response_data['quantityUnit']
            quantitaive_data = self.response_data['speciesQuantities']
            news_data = []
            totals_data = []
            for item in quantitaive_data:
                nd = item.get('newQuantities', None)
                td = item.get('totalQuantities', None)
                if nd:
                    news_data.append(nd)
                if td:
                    totals_data.append(td)

        return news_data, totals_data, unit_data


    def create_quantitative_dataframe(self, p_type):
        news_data, total_data, unit_data = self.preprocessing(p_type)

        species_dict = {}

        # Process 'news' data
        for item in news_data:
            species_type = f"{item['speciesName']} (Domestic)" if not item['isWild'] else f"{item['speciesName']} (Wild)"
            if species_type not in species_dict:
                species_dict[species_type] = {}
            species_dict[species_type]['news'] = {
                'New-Susceptible': item.get('susceptible', '-'),
                'New-Cases': item.get('cases', '-'),
                'New-Deaths': item.get('deaths', '-'),
                'New-Killed': item.get('killed', '-'),
                'New-Slaughtered': item.get('slaughtered', '-'),
                'New-Vaccinated': item.get('vaccinated', '-')
            }

        # Process 'totals' data
        for item in total_data:
            species_type = f"{item['speciesName']} (Domestic)" if not item['isWild'] else f"{item['speciesName']} (Wild)"
            if species_type not in species_dict:
                species_dict[species_type] = {}
            species_dict[species_type]['totals'] = {
                'Total-Susceptible': item.get('susceptible', '-'),
                'Total-Cases': item.get('cases', '-'),
                'Total-Deaths': item.get('deaths', '-'),
                'Total-Killed': item.get('killed', '-'),
                'Total-Slaughtered': item.get('slaughtered', '-'),
                'Total-Vaccinated': item.get('vaccinated', '-')
            }


        species_list = []
        category_list = []
        value_list = []
        for species, data in species_dict.items():
            new_list = [species]*12
            species_list = species_list + new_list
            new_data = data.get('news', {})
            if new_data != {}:
                category_list = category_list + list(new_data.keys())
                value_list = value_list + list(new_data.values())
            else:
                category_list = category_list + ['New-Susceptible', 'New-Cases', 'New-Deaths', 'New-Killed', 'New-Slaughtered', 'New-Vaccinated']
                value_list = value_list + ['-', '-', '-', '-', '-', '-']
            total_data = data.get('totals', {})
            if total_data != {}:
                category_list = category_list + list(total_data.keys())
                value_list = value_list + list(total_data.values())
            else:
                category_list = category_list + ['Total-Susceptible', 'Total-Cases', 'Total-Deaths', 'Total-Killed', 'Total-Slaughtered', 'Total-Vaccinated']
                value_list = value_list + ['-', '-', '-', '-', '-', '-']


        # 新しいDataFrameを作成
        df = pd.DataFrame()
        df['Item'] = ['Quantitative']*len(species_list)
        df['species'] = species_list
        df['category'] = category_list
        df['value'] = value_list

        return unit_data['id'], unit_data['translation'], df

    def get_species_unique(self):
        cells = self.ws.findall('Quantitative')
        species_unique = []
        for a in range(len(cells)):
            species_name = self.ws.cell(cells[a].row + 1, cells[a].col).value
            if species_name:
                species_unique.append(self.ws.cell(cells[a].row + 1, cells[a].col).value)
        return list(dict.fromkeys(species_unique))

    def _reorder_quant_values(self, quantitaive_df, report_species, species_unique):
        order, order_name, record = self._get_order_and_record(report_species, species_unique, quantitaive_df)

        after_Quant_value = self._get_after_quant_values(order, order_name, record, quantitaive_df, report_species)

        Quant_data = self._add_missing_species(after_Quant_value, species_unique, order)

        return Quant_data

    def _get_order_and_record(self, report_species, species_unique, quantitaive_df):
        order = []
        order_name = []
        record = []
        for species in report_species:
            try:
                order.append(species_unique.index(species))
                order_name.append(species)
            except ValueError:
                record.append(species)
                self.insert_new_cols(species, quantitaive_df)
        return order, order_name, record

    def _get_after_quant_values(self, order, order_name, record, quantitaive_df, report_species):
        order_sorted = sorted(order)
        after_Quant_value = pd.DataFrame()
        if order != order_sorted:
            for name in order_name:
                df_to_append = quantitaive_df[quantitaive_df['species'] == name]
                after_Quant_value = pd.concat([after_Quant_value, df_to_append], ignore_index=True)
        else:
            after_Quant_value = quantitaive_df

        if len(record) != 0:
            for name2 in record:
              df_to_append2 = quantitaive_df[quantitaive_df['species'] == name2]
              after_Quant_value = after_Quant_value[after_Quant_value['species'] != name2]
              after_Quant_value = pd.concat([after_Quant_value, df_to_append2], ignore_index=True)

        return after_Quant_value

    def _add_missing_species(self, Quant_df, species_unique, order):
        Qunat_list = Quant_df['value'].astype(str).tolist()
        diff_set = set([h for h in range(len(species_unique))]) - set(order)
        missing = sorted([diff for diff in diff_set])
        for c in range(len(missing)):
            Qunat_list[missing[c]:missing[c]] = ['-' for _ in range(12)]
        Quant_data = pd.DataFrame(Qunat_list, columns=['value'])
        return Quant_data

    def insert_new_cols(self, species, df):
        cells = self.ws.findall('Quantitative')
        Insert_col = df[df['species'] == species].drop('value', axis=1).astype(str).values.tolist()
        self.ws.insert_cols(Insert_col, col=cells[-1].col + 1)

    def update_ws_quant_data(self):
        if 'quantitativeData' in list(self.response_data.keys()):
            unitId, unitName, quantitaive_df = self.create_quantitative_dataframe("Summary")
        else:
            unitId, unitName, quantitaive_df = self.create_quantitative_dataframe("Detail")
        species_unique = self.get_species_unique()
        report_species = list(dict.fromkeys(quantitaive_df['species']))

        unit_data = {
            'Item': ['Quantitative', 'Quantitative'],
            'species': ['', ''],
            'category': ['id', 'Name'],
            'value': [unitId, unitName]
        }
        unit_df = pd.DataFrame(unit_data)

        if self.number == 0:
            Quant_data = quantitaive_df
            Quant_data = pd.concat([unit_df, Quant_data], axis = 0)
        elif species_unique != report_species:
            Quant_data = self._reorder_quant_values(quantitaive_df, report_species, species_unique)
            Quant_data = pd.concat([unit_df.filter(items=['value']), Quant_data], axis = 0)
        else:
            Quant_data = pd.concat([unit_df.filter(items=['value']), quantitaive_df.filter(items=['value'])], axis = 0)

        return Quant_data

### 統合

In [ ]:
class AutoReportScraper(CorrectList):
    def __init__(self, ws1, ws2, ss, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param):
        super().__init__(ws1, ws2, ss, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param)

    def scrape_reports_with_retry(self):
        n = 0
        prev_list = []
        same = 0
        while True:
          try:
              if n == 0:
                self.scrape()
              else:
                print("Restarted!")
                self.scrape()
          except Exception as e:
              n += 1
              print(f"An error happened: {e}")
              # エラーが発生した場合、現在の状態を取得して再実行する
              now_list = [CorrectList.nowPage, CorrectSummary.nowList, CorrectDetail.nowReport]
              print(now_list)
              self._get_current_state(CorrectList.nowPage, CorrectSummary.nowList, CorrectDetail.nowReport, CorrectList.finished, CorrectList.extracted_listdata)
              if (prev_list != []) and (set(prev_list) == set(now_list)):
                same += 1
                if same >= 5:
                  break
              if "429" in str(e):
                print("waiting 10 seconds ...")
                time.sleep(10)
                same -= 1
              prev_list = now_list
              time.sleep(2)
          else:
              break


    def _get_current_state(self, Page, List, Report, Finished, eventId_list):
        CorrectList.nowPage = Page
        CorrectSummary.nowList = List
        CorrectDetail.nowReport = Report
        CorrectList.finished = Finished
        CorrectList.extracted_listdata = eventId_list


In [ ]:
class ScrapingWAHIS():
    def __init__(self, area, disease, key = None, sheetname1 = None, sheetname2 = None, previous_date = None, from_date = None, to_date = None,
                 nowPage = None, nowList = None, nowReport = None):
        self.area = area
        self.disease = disease
        self.key = key
        self.sheetname1 = sheetname1
        self.sheetname2 = sheetname2
        self.pre_date = previous_date
        self.fd = from_date
        self.td = to_date

        CorrectList.update_classvalues(nowPage)
        CorrectSummary.update_classvalues(nowList)
        CorrectDetail.update_classvalues(nowReport)


    #@retry(retry=retry_if_exception_type((TimeoutError, APIError, HTTPError)))
    def run_scraping(self):
        MS = MakeSpreadSheet(self.area, self.disease, self.key, self.sheetname1, self.sheetname2)
        ss, ws1, ws2 = MS.make_spreadsheet()

        print("next")

        n = 0
        while True:
          try:
              if n == 0:
                  browser = HeadlessBrowser()
                  FQ_param, PQ_param, DQ_param = browser.visit_url(self.disease)
                  break
              else:
                  print("Restarted before scraping!")
                  browser = HeadlessBrowser()
                  FQ_param, PQ_param, DQ_param = browser.visit_url(self.disease)
                  break
          except Exception as e:
              n += 1
              print(f"An error happened in preparing: {e}")
              time.sleep(2)

        print("start")

        ARS = AutoReportScraper(ws1, ws2, ss, self.pre_date, self.fd, self.td, FQ_param, PQ_param, DQ_param)
        ARS.scrape_reports_with_retry()

In [ ]:
SW = ScrapingWAHIS("Asia/Tokyo", "Foot and mouth disease", key = None, sheetname1 = None, sheetname2 = None,
                   previous_date = None, from_date = None, to_date = None, nowPage = None, nowList = None, nowReport = None)
SW.run_scraping()

Foot and mouth disease_2024-07-29T21:32
next
start
0
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
1
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
2
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
3
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
4
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
5
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
6
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
7
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
8
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
9
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
10
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
11
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
12
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
13
https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en
14
http

### APIからの情報取得 Ver.2

In [ ]:
class CorrectDetail():
    '''
    class_variables = {
        'nowReport': None,
        'DQ_param': None,
    }

    @classmethod
    def update_class_variable(cls, variable_name, value):
        update_class_variable(cls, variable_name, value)
    '''

    @classmethod
    def update_classvalues(cls, nowReport=None):
        cls.nowReport = nowReport

    def __init__(self, df2, from_date, DQ_param):
        self.DQ_param = DQ_param
        self.df2 = df2
        self.from_date = from_date
        self.SR = SendRequest()

    def correct_detaildata(self, eventId, extracted_previewdata, list_number):
        if CorrectDetail.nowReport:
            number = CorrectDetail.nowReport
        else:
            CorrectDetail.nowReport = 0
            number = 0

        if not self.df2.empty:
            where_eve = self.df2.columns[self.df2.iloc[0] == str(eventId)]
        else:
            where_eve = pd.Index([])

        while number + 1 <= len(extracted_previewdata):
            response = self.SR.send_request_with_cookie(self.DQ_param, 'GET', [eventId, extracted_previewdata[number]])
            if response.status_code != 200:
                print(f"Error: Received status code {response.status_code}")
                break

            response_data = response.json()
            ref_col, check = self.check_date_and_ref_D(response_data)
            time.sleep(3)

            if check:
                processor1 = Qualitative_DataFormat(response_data, (list_number + number))
                Integrated_Info = processor1.integrate_information(eventId)

                processor2 = ControlDataFormat(response_data, self.df2, (list_number + number))
                Control_content, self.df2 = processor2.get_control_content()

                processor3 = QuantitativeDataFormat(response_data, self.df2, (list_number + number))
                Quant_data, self.df2 = processor3.update_ws_quant_data()

                Data = pd.concat([Integrated_Info, Control_content, Quant_data], axis=0).astype(str).reset_index(drop=True)
                print(Data)

                if ref_col is not None:
                    self.df2[ref_col] = Data['value'] if (list_number + number) == 0 else Data
                else:
                    if not where_eve.empty:
                        new_row_col = int(where_eve[-1]) + 1
                        self.df2 = pd.concat([self.df2.iloc[:, :new_row_col], Data, self.df2.iloc[:, new_row_col:]], axis=1, ignore_index=True).reset_index(drop=True)
                    else:
                        self.df2 = pd.concat([self.df2, Data], axis=1, ignore_index=True).reset_index(drop=True)

            print(self.df2)

            number += 1
            CorrectDetail.nowReport += 1
            time.sleep(3)

        print("Get all report in one list!")
        CorrectDetail.nowReport = None

    def check_date_and_ref_D(self, item):
        outbreak = pd.json_normalize(item['outbreak']).iloc[0]
        outbreakId = outbreak['outbreakId']
        Enddate = pd.to_datetime(outbreak['endDate']).date().strftime('%Y-%m-%d') if outbreak['endDate'] else '-'

        if not self.df2.empty:
            where_ref = self.df2.columns[self.df2.iloc[1] == str(outbreakId)]
        else:
            where_ref = pd.Index([])

        if self.from_date is not None:
            date_list = [Enddate, self.from_date]
            check = self.CD.narrow_down(date_list)
        else:
            check = False

        return where_ref[0] if not where_ref.empty else None, (not check)


In [ ]:
class CorrectSummary(CorrectDetail):
    @classmethod
    def update_classvalues(cls, nowList=None):
        cls.nowList = nowList

    def __init__(self, df1, df2, from_date, PQ_param, DQ_param):
        self.PQ_param = PQ_param
        self.df1 = df1
        self.from_date = from_date
        self.SR = SendRequest()
        super().__init__(df2, from_date, DQ_param)

    def correct_previewdata(self, extracted_listdata):
        if CorrectSummary.nowList:
            number = CorrectSummary.nowList
        else:
            CorrectSummary.nowList = 0
            number = 0

        while number + 1 <= len(extracted_listdata):
            response = self.SR.send_request_with_cookie(self.PQ_param, 'GET', [extracted_listdata[number]['eventId']])
            if response.status_code != 200:
                print(f"Error: Received status code {response.status_code}")
                break
            else:
                print("Gotcha!")

            response_data = response.json()
            ref_col, check = self.check_date_and_ref_S(response_data)

            if check:
                processor1 = Qualitative_DataFormat(response_data, number)
                Integrated_Info = processor1.integrate_information()

                processor2 = ControlDataFormat(response_data, self.df1, number)
                Control_content, self.df1 = processor2.get_control_content()

                processor3 = QuantitativeDataFormat(response_data, self.df1, number)
                Quant_data, self.df1 = processor3.update_ws_quant_data()

                Data = pd.concat([Integrated_Info, Control_content, Quant_data], axis=0).astype(str).reset_index(drop=True)

                if ref_col is not None:
                    self.df1[ref_col] = Data['value'] if number == 0 else Data
                else:
                    self.df1 = pd.concat([self.df1, Data], axis=1, ignore_index=True).reset_index(drop=True)
                print(self.df1)

            outbreak_df = pd.json_normalize(response_data['outbreaks'])
            outbreak_ids = outbreak_df['outbreakId'].tolist()
            self.correct_detaildata(extracted_listdata[number]['eventId'], outbreak_ids, number)

            number += 1
            CorrectSummary.nowList += 1

    def check_date_and_ref_S(self, item):
        events = pd.json_normalize(item['event']).iloc[0]
        eventId = events['eventId']
        Enddate = pd.to_datetime(events['endedOn']).date().strftime('%Y-%m-%d') if events['endedOn'] else '-'

        if not self.df1.empty:
            where_ref = self.df1.columns[self.df1.iloc[0] == str(eventId)]
        else:
            where_ref = pd.Index([])

        if self.from_date is not None:
            date_list = [Enddate, self.from_date]
            check = self.CD.narrow_down(date_list)
        else:
            check = False

        return where_ref[0] if not where_ref.empty else None, (not check)


In [ ]:
class CorrectList(CorrectSummary):
    @classmethod
    def update_classvalues(cls, nowPage=None, finished=False, extracted_listdata=[]):
        cls.nowPage = nowPage
        cls.finished = finished
        cls.extracted_listdata = extracted_listdata

    def __init__(self, df1, df2, previous_date, from_date, to_date, FQ_param, PQ_param, DQ_param):
        self.FQ_param = FQ_param
        self.to_date = to_date
        self.CD = CompareDate()
        self.SR = SendRequest()
        self.pre_date = previous_date
        self.df1 = df1
        self.df2 = df2
        super().__init__(df1, df2, from_date, PQ_param, DQ_param)

    def scrape(self):
        if not CorrectList.finished:
            self.correct_listdata()
        self.correct_previewdata(CorrectList.extracted_listdata)
        self.df1.to_csv('Summary.csv', index=False)
        self.df2.to_csv('Summary.csv', index=False)

    def correct_listdata(self):
        # リクエストを送信し、結果を表示
        if CorrectList.nowPage:
            self.FQ_param["body"]["pageNumber"] = CorrectList.nowPage
        else:
            CorrectList.nowPage = 0

        while True:
            print(CorrectList.nowPage)

            response = self.SR.send_request_with_cookie(self.FQ_param, 'POST')
            if response.status_code != 200:
                print(f"Error: Received status code {response.status_code}")
                break

            response_data = response.json()

            if not response_data.get('list'):
                print("No more data available.")
                CorrectList.finished = True
                break

            # 必要な情報を抽出してリストに整理
            for item in response_data['list']:
                check = self.check_date(item)

                if check:
                    CorrectList.extracted_listdata.append({
                        'eventId': item['eventId']
                    })

            # 次のページへ
            self.FQ_param["body"]["pageNumber"] += 1
            CorrectList.nowPage += 1

    def check_date(self, item):
        Update_date = pd.to_datetime(item['submissionDate']).date() if item['submissionDate'] is not None else '-'
        Start_date = pd.to_datetime(item['eventStartDate']).date() if item['eventStartDate'] is not None else '-'

        if self.pre_date is not None:
            date_list = [Update_date, self.pre_date]
            checkA = self.CD.narrow_down(date_list)
        else:
            checkA = False

        if self.to_date is not None:
            date_list2 = [self.to_date, Start_date]
            checkB = self.CD.narrow_down(date_list2)
        else:
            checkB = False

        return not (checkA or checkB)


### データ整理クラス関数 ver.2

In [ ]:
class Qualitative_DataFormat:
    def __init__(self, response_data, number):
        self.response_data = response_data
        self.number = number

    def create_common_info_S(self):
        events = self.response_data['event']
        sources = pd.json_normalize(self.response_data['sources'])
        methods = pd.json_normalize(self.response_data['methods'])
        laboratoryTests = pd.json_normalize(self.response_data['laboratoryTests'])

        common_info = {
            'eventId': events['eventId'],
            'country.areaId': events['country']['areaId'],
            'country.name': events['country']['name'],
            'country.isoCode': events['country']['isoCode'],
            'startedOn': pd.to_datetime(events['startedOn']).date() if events['startedOn'] else '-',
            'lastOccurrence': pd.to_datetime(events['lastOccurrence']).date() if events['lastOccurrence'] else '-',
            'endedOn': pd.to_datetime(events['endedOn']).date() if events['endedOn'] else '-',
            'eventStatus.id': events['eventStatus']['id'],
            'eventStatus.translation': events['eventStatus']['translation'],
            'isAquatic': events['isAquatic'],
            'disease.diseaseId': events['disease']['diseaseId'],
            'disease.name': events['disease']['name'],
            'subType.diseaseId': events['subType']['disease']['diseaseId'] if events['subType']['disease'] else 'Pending',
            'subType.name': events['subType']['disease']['name'] if events['subType']['disease'] else 'Pending',
            'reason.id': events['reason']['id'],
            'reason.translation': events['reason']['translation'],
            'clinicalSigns': events['clinicalSigns'],
            'sources.sourceId': sources['source.sourceId'].tolist(),
            'sources.name': sources['source.name'].tolist(),
            'eventComment': events['eventComment']['translation'] if events['eventComment'] else 'No epidemiological comment',
            'methods.id': methods['nature.id'].tolist(),
            'methods.translation': methods['nature.translation'].tolist(),
            'laboratoryTests.testName': laboratoryTests['testName'].tolist(),
            'confirmOn': pd.to_datetime(events['confirmOn']).date() if events['confirmOn'] else '-',
            'laboratoryTests.result': laboratoryTests['result'].tolist(),
        }

        return common_info

    def create_common_info_D(self, eventId):
        outbreak = pd.json_normalize(self.response_data['outbreak']).iloc[0]
        adminDivisions = self.response_data['adminDivisions']
        diagnosticMethods = pd.json_normalize(self.response_data['diagnosticMethods'])

        common_info = {
            'eventId': eventId,
            'outbreakId': outbreak['outbreakId'],
            'areaId': outbreak['areaId'],
            'latitude': outbreak['latitude'],
            'longtitude': outbreak['longitude'],
            'epiUnitType_id': outbreak['epiUnitType.id'],
            'epiUnitType_translation': outbreak['epiUnitType.translation'],
            'startDate': pd.to_datetime(outbreak['startDate']).date() if outbreak['startDate'] else '-',
            'endDate': pd.to_datetime(outbreak['endDate']).date() if outbreak['endDate'] else '-',
            'diagnosticMethods.id': diagnosticMethods['nature.id'].tolist() if not diagnosticMethods.empty else '-',
            'diagnosticMethods.translation': diagnosticMethods['nature.translation'].tolist() if not diagnosticMethods.empty else '-',
        }

        # 管理区分の情報を初期化し、存在する場合は更新
        for i in range(3):
            common_info[f'adminDivisions.{i+1}.id'] = '-'
            common_info[f'adminDivisions.{i+1}.name'] = '-'
        for item in adminDivisions:
            L = item['adminLevel']
            common_info[f'adminDivisions.{L}.id'] = item['areaId']
            common_info[f'adminDivisions.{L}.name'] = item['name']

        return common_info

    def integrate_information(self, eventId=None):
        if 'event' in self.response_data:
            info = self.create_common_info_S()
        elif 'outbreak' in self.response_data:
            info = self.create_common_info_D(eventId)

        df = pd.DataFrame({
            'Item': ['Summary'] * len(info),
            'species': [''] * len(info),
            'category': list(info.keys()),
            'value': list(info.values())
        })

        if self.number == 0:
            Integrated_Info = df
        else:
            Integrated_Info = df[['value']]

        return Integrated_Info


In [ ]:
class ControlDataFormat:
    def __init__(self, response_data, df, number):
        self.response_data = response_data
        self.df = df
        self.number = number

    def extract_measures_info_S(self, measures_data):
        measures_name = []
        domestic_value = []
        wild_value = []

        for index, measure in measures_data.iterrows():
            measures_name.append(measure['measure.name'])

            domestic_status = '-'
            wild_status = '-'

            category_original = measure['measure.category.original'].lower()
            if 'domestic' in category_original:
                domestic_status = measure['status.translation']
            elif 'wild' in category_original:
                wild_status = measure['status.translation']

            domestic_value.append(domestic_status)
            wild_value.append(wild_status)

        df = pd.DataFrame()
        df['Item'] = ['Control Measure'] * len(measures_name) * 2
        df['species'] = [item for item in ['Domestic', 'Wild'] for _ in range(len(measures_name))]
        df['category'] = measures_name * 2
        df['value'] = domestic_value + wild_value
        report_cont = list(dict.fromkeys(measures_name))

        return df, report_cont

    def extract_measures_info_D(self):
        measure_name = []
        measure_apply = []
        measure = self.response_data['controlMeasures']
        add_measures = self.response_data['additionalMeasures']

        for item in measure:
            measure_name.append(item['measure']['name'])

            default_status = '-'
            if item['status']['translation'] == 'Applied':
                measure_apply.append('Applied')
            else:
                measure_apply.append(default_status)

        for item in add_measures:
            measure_name.append(item['name'])
            measure_apply.append('Applied')

        df = pd.DataFrame()
        df['Item'] = ['Control Measure'] * len(measure_apply)
        df['species'] = ['' for _ in range(len(measure_apply))]
        df['category'] = measure_name
        df['value'] = measure_apply
        report_cont = list(dict.fromkeys(df['category']))

        return df, report_cont

    def get_cont_unique(self):
        if not self.df.empty:
            cont_unique = self.df[self.df[0] == 'Control Measure'][2].unique().tolist()
        else:
            cont_unique = []
        return cont_unique

    def update_df_cont_data(self, cont_unique, report_cont, control_df):

        order, order_name, record = self.get_order_and_record(cont_unique, report_cont, control_df)

        Cont_df = self.get_after_cont_content(order, order_name, record, report_cont, cont_unique, control_df)

        Cont_data = self.add_missing_species(Cont_df, cont_unique, order)

        return Cont_data

    def get_order_and_record(self, cont_unique, report_cont, control_df):
        order = []
        order_name = []
        record = []
        insert_ver = True if cont_unique == [] else False

        for mea in report_cont:
            try:
                order.append(cont_unique.index(mea))
                order_name.append(mea)
            except ValueError:
                record.append(mea)
                self.insert_new_cols(mea, control_df, insert_ver)

        return order, order_name, record

    def insert_new_cols(self, cont_category, df, insert_ver):
        new_rows = df[df['category'] == cont_category].drop('value', axis=1).astype(str)
        new_rows.columns = [0, 1, 2]
        if insert_ver:
            summary_row_index = self.df[self.df[0] == 'Summary'].index.values[-1] + 1
            self.df = pd.concat([self.df.iloc[:summary_row_index], new_rows, self.df.iloc[summary_row_index:]]).reset_index(drop=True)
        else:
            control_measure_row_index = self.df[self.df[0] == 'Control Measure'].index.values[-1] + 1

            if 'event' in self.response_data.keys():
                domestic = new_rows[new_rows['species'] == 'Domestic']
                wild = new_rows[new_rows['species'] == 'Wild']

                self.df = pd.concat([self.df.iloc[:control_measure_row_index], domestic, self.df.iloc[control_measure_row_index:]], ignore_index=True)

                wild_insert_row_index = control_measure_row_index + len(domestic)
                self.df = pd.concat([self.df.iloc[:wild_insert_row_index], wild, self.df.iloc[wild_insert_row_index:]], ignore_index=True)
            else:
                self.df = pd.concat([self.df.iloc[:control_measure_row_index], new_rows, self.df.iloc[control_measure_row_index:]], ignore_index=True)


    def get_after_cont_content(self, order, order_name, record, report_cont, cont_unique, control_df):
        order_sorted = sorted(order)
        after_Cont_content = pd.DataFrame()
        if order != order_sorted:
            for name in order_name:
                df_to_append = control_df[control_df['category'] == name]
                after_Cont_content = pd.concat([after_Cont_content, df_to_append], ignore_index=True)
        else:
            after_Cont_content = control_df

        if len(record) != 0:
            for name2 in record:
                df_to_append2 = control_df[control_df['category'] == name2]
                after_Cont_content = after_Cont_content[after_Cont_content['category'] != name2]
                after_Cont_content = pd.concat([after_Cont_content, df_to_append2], ignore_index=True)

        return after_Cont_content

    def add_missing_species(self, Cont_df, cont_unique, order):
        if len(cont_unique) != len(order):
            Cont_list = Cont_df['value'].astype(str).tolist()
            diff_set = set([h for h in range(len(cont_unique))]) - set(order)
            missing = sorted([diff for diff in diff_set])

            if Cont_list == [] and 'event' in self.response_data.keys():
                Cont_list = np.full(len(missing) * 2, '-', dtype=object)
            elif Cont_list == []:
                Cont_list = np.full(len(missing), '-', dtype=object)
            else:
                for c in range(len(missing)):
                    Cont_list = np.insert(np.reshape(Cont_list, (2, -1)), missing[c], np.array(['-' for _ in range(2)], dtype=object), axis=1).ravel()

            return pd.DataFrame(Cont_list.tolist(), columns=['value'])
        else:
            return Cont_df.filter(items=['value'])

    def get_control_content(self):
        if 'measures' in list(self.response_data.keys()):
            measures_data = pd.json_normalize(self.response_data['measures'])
            measures_df, report_cont = self.extract_measures_info_S(measures_data)
        else:
            measures_df, report_cont = self.extract_measures_info_D()

        cont_unique = self.get_cont_unique()

        if self.number == 0:
            Control_content = measures_df
        elif cont_unique != report_cont:
            Control_content = self.update_df_cont_data(cont_unique, report_cont, measures_df)
        else:
            Control_content = measures_df[['value']]

        return Control_content, self.df


In [ ]:
class QuantitativeDataFormat:
    def __init__(self, response_data, df, number):
        self.response_data = response_data
        self.df = df
        self.number = number

    def preprocessing(self, p_type):
        if p_type == 'Summary':
            quantitative_data = self.response_data['quantitativeData']
            news_data = quantitative_data.get('news', [])
            totals_data = quantitative_data.get('totals', [])
            unit_data = quantitative_data.get('unit', {})
        elif p_type == 'Detail':
            unit_data = self.response_data['quantityUnit']
            quantitaive_data = self.response_data['speciesQuantities']
            news_data, totals_data = [], []
            for item in quantitaive_data:
                nd = item.get('newQuantities', None)
                td = item.get('totalQuantities', None)
                if nd:
                    news_data.append(nd)
                if td:
                    totals_data.append(td)

        return news_data, totals_data, unit_data

    def create_quantitative_dataframe(self, p_type):
        news_data, total_data, unit_data = self.preprocessing(p_type)

        species_dict = {}

        # Process 'news' data
        for item in news_data:
            species_type = f"{item['speciesName']} (Domestic)" if not item['isWild'] else f"{item['speciesName']} (Wild)"
            if species_type not in species_dict:
                species_dict[species_type] = {}
            species_dict[species_type]['news'] = {
                'New-Susceptible': item.get('susceptible', '-'),
                'New-Cases': item.get('cases', '-'),
                'New-Deaths': item.get('deaths', '-'),
                'New-Killed': item.get('killed', '-'),
                'New-Slaughtered': item.get('slaughtered', '-'),
                'New-Vaccinated': item.get('vaccinated', '-')
            }

        # Process 'totals' data
        for item in total_data:
            species_type = f"{item['speciesName']} (Domestic)" if not item['isWild'] else f"{item['speciesName']} (Wild)"
            if species_type not in species_dict:
                species_dict[species_type] = {}
            species_dict[species_type]['totals'] = {
                'Total-Susceptible': item.get('susceptible', '-'),
                'Total-Cases': item.get('cases', '-'),
                'Total-Deaths': item.get('deaths', '-'),
                'Total-Killed': item.get('killed', '-'),
                'Total-Slaughtered': item.get('slaughtered', '-'),
                'Total-Vaccinated': item.get('vaccinated', '-')
            }

        species_list = []
        category_list = []
        value_list = []
        for species, data in species_dict.items():
            species_list.extend([species]*12)
            category_list.extend(data.get('news', {}).keys() or ['New-Susceptible', 'New-Cases', 'New-Deaths', 'New-Killed', 'New-Slaughtered', 'New-Vaccinated'])
            value_list.extend(data.get('news', {}).values() or ['-', '-', '-', '-', '-', '-'])
            category_list.extend(data.get('totals', {}).keys() or ['Total-Susceptible', 'Total-Cases', 'Total-Deaths', 'Total-Killed', 'Total-Slaughtered', 'Total-Vaccinated'])
            value_list.extend(data.get('totals', {}).values() or ['-', '-', '-', '-', '-', '-'])

        # Create DataFrame
        df = pd.DataFrame({
            'Item': ['Quantitative']*len(species_list),
            'species': species_list,
            'category': category_list,
            'value': value_list
        })

        return unit_data.get('id', '-'), unit_data.get('translation', '-'), df

    def get_species_unique(self):
        if not self.df.empty:
            species_unique = self.df[self.df[0] == 'Quantitative'][1].unique().tolist()
            species_unique.remove("")
        else:
            species_unique = []
        return species_unique

    def _reorder_quant_values(self, quantitaive_df, report_species, species_unique):
        order, order_name, record = self._get_order_and_record(report_species, species_unique, quantitaive_df)

        after_Quant_value = self._get_after_quant_values(order, order_name, record, quantitaive_df)

        Quant_data = self._add_missing_species(after_Quant_value, species_unique, order)

        return Quant_data

    def _get_order_and_record(self, report_species, species_unique, quantitaive_df):
        order = []
        order_name = []
        record = []
        for species in report_species:
            try:
                order.append(species_unique.index(species))
                order_name.append(species)
            except ValueError:
                record.append(species)
                self.insert_new_rows(species, quantitaive_df)
        return order, order_name, record

    def _get_after_quant_values(self, order, order_name, record, quantitaive_df):
        order_sorted = sorted(order)
        after_Quant_value = pd.DataFrame()
        if order != order_sorted:
            for name in order_name:
                df_to_append = quantitaive_df[quantitaive_df['species'] == name]
                after_Quant_value = pd.concat([after_Quant_value, df_to_append], ignore_index=True)
        else:
            after_Quant_value = quantitaive_df

        if record:
            for name2 in record:
                df_to_append2 = quantitaive_df[quantitaive_df['species'] == name2]
                after_Quant_value = after_Quant_value[after_Quant_value['species'] != name2]
                after_Quant_value = pd.concat([after_Quant_value, df_to_append2], ignore_index=True)

        return after_Quant_value

    def _add_missing_species(self, Quant_df, species_unique, order):
        Qunat_list = Quant_df['value'].astype(str).tolist()
        diff_set = set(range(len(species_unique))) - set(order)
        missing = sorted(diff for diff in diff_set)
        for c in range(len(missing)):
            Qunat_list[missing[c]:missing[c]] = ['-' for _ in range(12)]
        Quant_data = pd.DataFrame(Qunat_list, columns=['value'])
        return Quant_data

    def insert_new_rows(self, species, df):
        new_row = df[df['species'] == species].drop('value', axis=1).reset_index(drop=True)
        new_rows.columns = [0, 1, 2]
        self.df = pd.concat([self.df.iloc[:self.df[self.df[0] == 'Quantitative'].index.values[-1] + 1], new_row, self.df.iloc[self.df[self.df[0] == 'Quantitative'].index.values[-1] + 1:]]).reset_index(drop=True)

    def update_ws_quant_data(self):
        if 'quantitativeData' in self.response_data:
            unitId, unitName, quantitaive_df = self.create_quantitative_dataframe("Summary")
        else:
            unitId, unitName, quantitaive_df = self.create_quantitative_dataframe("Detail")

        species_unique = self.get_species_unique()
        report_species = quantitaive_df['species'].unique().tolist()

        unit_data = {
            'Item': ['Quantitative', 'Quantitative'],
            'species': ['', ''],
            'category': ['id', 'Name'],
            'value': [unitId, unitName]
        }
        unit_df = pd.DataFrame(unit_data)

        if self.number == 0:
            Quant_data = pd.concat([unit_df, quantitaive_df], axis=0)
        elif species_unique != report_species:
            Quant_data = self._reorder_quant_values(quantitaive_df, report_species, species_unique)
            Quant_data = pd.concat([unit_df[['value']], Quant_data], axis=0)
        else:
            Quant_data = pd.concat([unit_df[['value']], quantitaive_df[['value']]], axis=0)

        return Quant_data, self.df


In [ ]:
df2 = pd.read_csv("/content/Detail.csv")
df2_ = df2.T.reset_index()

df2_subset = df2_.iloc[3:, :]
df2_subset[[0, 1]] = df2_subset[[0, 1]].astype(int)
# 0列目（ここでは'A'列）を基準にソート
df2_sorted_subset = df2_subset.sort_values(by=[0, 1], ascending=[False, False])

# ソートされた部分を元のデータフレームに戻す
df2_.iloc[3:, :] = df2_sorted_subset
df2_[[0,1]] = df2_[[0,1]].astype(str)
print(df2_.iloc[:5, :5])
df2_ = df2_.drop(columns=['index'], axis=1).reset_index()
df2_.to_csv('Detail_.csv', index=False)
print(df2_.iloc[:5, :5])

<ipython-input-80-0ea3e7eba650>:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,26

  index        0           1        2         3
0     0  Summary     Summary  Summary   Summary
1     1      nan         nan      NaN       NaN
2     2  eventId  outbreakId   areaId  latitude
3   197     5735      135852   214828     -32.8
4   198     5735      135851   214797     -32.8
   index        0           1        2         3
0      0  Summary     Summary  Summary   Summary
1      1      nan         nan      NaN       NaN
2      2  eventId  outbreakId   areaId  latitude
3      3     5735      135852   214828     -32.8
4      4     5735      135851   214797     -32.8


## その他

### Cookie取得

In [ ]:
# クッキーを生成する関数
def generate_unique_cookie():
    cookie_value = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
    cookie_jar = CookieJar()
    cookie_jar.set_cookie(requests.cookies.create_cookie(domain='wahis.woah.org', name='_custom_cookie', value=cookie_value))
    return cookie_jar

# クッキーを作成
cookies = generate_unique_cookie()

# cURLリクエストを送信する関数
def send_request_with_cookie():
    url = 'https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en'
    headers = {
        'accept': 'application/json',
        'content-type': 'application/json'
    }
    data = {
        "eventIds": [],
        "reportIds": [],
        "countries": [],
        "firstDiseases": [437],
        "secondDiseases": [],
        "typeStatuses": [],
        "reasons": [],
        "eventStatuses": [],
        "reportTypes": [],
        "reportStatuses": [],
        "eventStartDate": None,
        "submissionDate": None,
        "animalTypes": [],
        "sortColumn": "submissionDate",
        "sortOrder": "desc",
        "pageSize": 100,
        "pageNumber": 0
    }
    response = requests.post(url, headers=headers, json=data, cookies=cookies)
    return response.json()

# リクエストを送信し、結果を表示
response = send_request_with_cookie()
print(response)


{'list': [{'reportId': 168176, 'eventId': 5658, 'country': 'South Africa', 'disease': 'Foot and mouth disease virus (Inf. with) ', 'subType': 'SAT 3', 'eventStartDate': '2024-04-14T00:00:00.000+00:00', 'eventStatus': 'On-going', 'reason': 'First occurrence in a zone or a compartment', 'reportType': 'FUR', 'reportStatus': 'Validated', 'submissionDate': '2024-07-10T12:12:54.575+00:00', 'reportNumber': 7, 'isAquatic': False}, {'reportId': 168003, 'eventId': 5735, 'country': 'South Africa', 'disease': 'Foot and mouth disease virus (Inf. with) ', 'subType': 'SAT 2', 'eventStartDate': '2024-04-30T00:00:00.000+00:00', 'eventStatus': 'On-going', 'reason': 'New strain in a zone or a compartment', 'reportType': 'IN', 'reportStatus': 'Validated', 'submissionDate': '2024-07-05T10:37:56.349+00:00', 'reportNumber': 0, 'isAquatic': False}, {'reportId': 167158, 'eventId': 5487, 'country': 'South Africa', 'disease': 'Foot and mouth disease virus (Inf. with) ', 'subType': 'SAT 1', 'eventStartDate': '202

### 確認

In [ ]:
import json

# 例としてJSONデータを変数に格納
data = '''{
    "list": [
        {
            "reportId": 168176,
            "eventId": 5658,
            "country": "South Africa",
            "disease": "Foot and mouth disease virus (Inf. with)",
            "subType": "SAT 3",
            "eventStartDate": "2024-04-14T00:00:00.000+00:00",
            "eventStatus": "On-going",
            "reason": "First occurrence in a zone or a compartment",
            "reportType": "FUR",
            "reportStatus": "Validated",
            "submissionDate": "2024-07-10T12:12:54.575+00:00",
            "reportNumber": 7,
            "isAquatic": False
        }
        // 他のレコードが続く
    ]
}'''

# JSONデータをPythonの辞書に変換
data_dict = json.loads(data)

# 必要な情報を抽出
for item in data_dict["list"]:
    event_id = item["eventId"]
    country = item["country"]
    event_start_date = item["eventStartDate"]
    print(f"Event ID: {event_id}, Country: {country}, Event Start Date: {event_start_date}")


### 関数草案

In [ ]:
# cURLリクエストを送信する関数
def send_request_with_cookie(query_param):
    url = query_param["url"]
    headers = {
        'accept': 'application/json',
        'content-type': 'application/json'
    }
    '''
    cookies = {}
    for cookie in query_param["headers"]["cookie"].split('; '):
        name, value = cookie.split('=')
        cookies[name] = value
    '''
    data = query_param["body"]
    #response = requests.post(url, headers=headers, json=data, cookies=cookies)
    response = requests.post(url, headers=headers, json=data)
    return response

# リクエストを送信し、結果を表示
extracted_data = []
while True:
    response = send_request_with_cookie(FQ_param)
    if response.status_code != 200:
          print(f"Error: Received status code {response.status_code}")
          break

    response_data = response.json()

    if not response_data.get('list'):
        print("No more data available.")
        break

    # 必要な情報を抽出してリストに整理
    for item in response_data['list']:
        extracted_data.append([
            item['reportId'],
            item['eventId'],
            item['country'],
            item['disease'],
            item['subType'],
            item['eventStartDate'],
            item['eventStatus'],
            item['reason'],
            item['reportType'],
            item['reportStatus'],
            item['submissionDate'],
            item['reportNumber'],
            item['isAquatic']
        ])

    # 次のページへ
    FQ_param["body"]["pageNumber"] += 1


for data in extracted_data:
    print(data)


No more data available.
[168176, 5658, 'South Africa', 'Foot and mouth disease virus (Inf. with) ', 'SAT 3', '2024-04-14T00:00:00.000+00:00', 'On-going', 'First occurrence in a zone or a compartment', 'FUR', 'Validated', '2024-07-10T12:12:54.575+00:00', 7, False]
[168003, 5735, 'South Africa', 'Foot and mouth disease virus (Inf. with) ', 'SAT 2', '2024-04-30T00:00:00.000+00:00', 'On-going', 'New strain in a zone or a compartment', 'IN', 'Validated', '2024-07-05T10:37:56.349+00:00', 0, False]
[167158, 5487, 'South Africa', 'Foot and mouth disease virus (Inf. with) ', 'SAT 1', '2024-01-04T00:00:00.000+00:00', 'Resolved', 'Recurrence of an eradicated disease', 'FUR', 'Validated', '2024-06-05T06:35:10.380+00:00', 6, False]
[167564, 3738, 'South Africa', 'Foot and mouth disease virus (Inf. with) ', 'SAT 2', '2021-05-26T00:00:00.000+00:00', 'On-going', 'Recurrence of an eradicated disease', 'FUR', 'Validated', '2024-06-05T06:01:44.438+00:00', 39, False]
[167157, 4368, 'South Africa', 'Foot a

In [ ]:
def create_common_info(response_data):
    events = pd.json_normalize(response_data['event']).iloc[0]
    #reports = pd.json_normalize(response_data['report']).iloc[0]
    sources = pd.json_normalize(response_data['sources'])
    methods = pd.json_normalize(response_data['methods'])
    laboratoryTests = pd.json_normalize(response_data['laboratoryTests'])

    common_info = {
        'eventId': events['eventId'],
        'country.areaId': events['country.areaId'],
        'country.name': events['country.name'],
        'country.isoCode': events['country.isoCode'],
        'startedOn': pd.to_datetime(events['startedOn']).date() if events['startedOn'] is not None else '-',
        'lastOccurrence': pd.to_datetime(events['lastOccurrence']).date() if events['lastOccurrence'] is not None else '-',
        'endedOn': pd.to_datetime(events['endedOn']).date() if events['endedOn'] is not None else '-',
        'eventStatus.id': events['eventStatus.id'],
        'eventStatus.translation': events['eventStatus.translation'],
        'isAquatic': events['isAquatic'],
        'disease.diseaseId': events['disease.diseaseId'],
        'disease.name': events['disease.name'],
        'subType.diseaseId': events['subType.disease.diseaseId'],
        'subType.name': events['subType.disease.name'],
        'reason.id': events['reason.id'],
        'reason.translation': events['reason.translation'],
        'clinicalSigns': events['clinicalSigns'],
        'sources.sourceId': sources['source.sourceId'].tolist(),
        'sources.name': sources['source.name'].tolist(),
        'eventComment': events['eventComment.translation'],
        'methods.id': methods['nature.id'].tolist(),
        'methods.translation': methods['nature.translation'].tolist(),
        'laboratoryTests.testName': laboratoryTests['testName'].tolist(),
        'confirmOn': pd.to_datetime(events['confirmOn']).date() if events['confirmOn'] is not None else '-',
        'laboratoryTests.result': laboratoryTests['result'].tolist(),
        # 他に必要な情報を追加する
    }

    df = pd.DataFrame()
    df['Item'] = ['Summary']*len(common_info)
    df['species'] = ['']*len(common_info)
    df['category'] = list(common_info.keys())
    df['value'] = list(common_info.values())

    return df.transpose()

In [ ]:
def create_quantitative_dataframe(quantitative_data):
    news_data = quantitative_data.get('news', [])[0]
    totals_data = quantitative_data.get('totals', [])[0]

    species_dict = {}

    # Process 'news' data
    for item in news_data:
        species_type = f"{item['speciesName']} (Domestic)" if not item['isWild'] else f"{item['speciesName']} (Wild)"
        if species_type not in species_dict:
            species_dict[species_type] = {}
        species_dict[species_type]['news'] = {
            'New-Susceptible': item.get('susceptible', '-'),
            'New-Cases': item.get('cases', '-'),
            'New-Deaths': item.get('deaths', '-'),
            'New-Killed': item.get('killed', '-'),
            'New-Slaughtered': item.get('slaughtered', '-'),
            'New-Vaccinated': item.get('vaccinated', '-')
        }

    # Process 'totals' data
    for item in totals_data:
        species_type = f"{item['speciesName']} (Domestic)" if not item['isWild'] else f"{item['speciesName']} (Wild)"
        if species_type not in species_dict:
            species_dict[species_type] = {}
        species_dict[species_type]['totals'] = {
            'Total-Susceptible': item.get('susceptible', '-'),
            'Total-Cases': item.get('cases', '-'),
            'Total-Deaths': item.get('deaths', '-'),
            'Total-Killed': item.get('killed', '-'),
            'Total-Slaughtered': item.get('slaughtered', '-'),
            'Total-Vaccinated': item.get('vaccinated', '-')
        }

    species_list = []
    category_list = []
    value_list = []
    for species, data in species_dict.items():
        new_list = [species]*12
        species_list = species_list + new_list
        new_data = data.get('news', {})
        if new_data != {}:
            category_list = category_list + list(new_data.keys())
            value_list = value_list + list(new_data.values())
        else:
            category_list = category_list + ['New-Susceptible', 'New-Cases', 'New-Deaths', 'New-Killed', 'New-Slaughtered', 'New-Vaccinated']
            value_list = value_list + ['-', '-', '-', '-', '-', '-']
        total_data = data.get('totals', {})
        if total_data != {}:
            category_list = category_list + list(total_data.keys())
            value_list = value_list + list(total_data.values())
        else:
            category_list = category_list + ['Total-Susceptible', 'Total-Cases', 'Total-Deaths', 'Total-Killed', 'Total-Slaughtered', 'Total-Vaccinated']
            value_list = value_list + ['-', '-', '-', '-', '-', '-']


    # 新しいDataFrameを作成
    df = pd.DataFrame()
    df['Item'] = ['Quantative']*len(species_list)
    df['species'] = species_list
    df['category'] = category_list
    df['value'] = value_list

    return df.transpose()

In [ ]:
def extract_measures_info(measures_data):
    measures_name = []
    domestic_value = []
    wild_value = []

    for index, measure in measures_data.iterrows():
        measures_name.append(measure['measure.name'])

        # デフォルトの値を設定
        domestic_status = '-'
        wild_status = '-'

        category_original = measure['measure.category.original'].lower()
        if 'domestic' in category_original:
            domestic_status = measure['status.translation']
        elif 'wild' in category_original:
            wild_status = measure['status.translation']

        domestic_value.append(domestic_status)
        wild_value.append(wild_status)

    df = pd.DataFrame()
    df['Item'] = ['Control Measure']*len(measures_name)*2
    df['species'] = [item for item in ['Domestic', 'Wild'] for _ in range(len(measures_name))]
    df['category'] = measures_name*2
    df['value'] = domestic_value + wild_value

    return df.transpose()

In [ ]:
# リクエストを送信し、結果を表示
extracted_detaildata = []
number = 0
while True:
    print(number)
    response = send_request_with_cookie(PQ_param, [extracted_listdata[number]['eventId'], extracted_listdata[number]])
    if response.status_code != 200:
          print(f"Error: Received status code {response.status_code}")
          break

    response_data = response.json()

    '''
    if not response_data.get('list'):
        print("No more data available.")
        break
    '''

    # 必要な情報を抽出してリストに整理
    for item in response_data['list']:
        extracted_detaildata.append([
            item['reportId'],
            item['eventId'],
            item['country'],
            item['disease'],
            item['subType'],
            item['eventStartDate'],
            item['eventStatus'],
            item['reason'],
            item['reportType'],
            item['reportStatus'],
            item['submissionDate'],
            item['reportNumber'],
            item['isAquatic']
        ])

    # 次のリストへ
    number += 1


for data in extracted_detaildata:
    print(data)

In [ ]:
# リクエストを送信し、結果を表示
extracted_outbreakdata = []
number = 0
while number+1 <= len(extracted_listdata):
    print(number)
    response = send_request_with_cookie(PQ_param, 'GET', [extracted_listdata[number]['eventId']])
    if response.status_code != 200:
          print(f"Error: Received status code {response.status_code}")
          break

    outbreak_df = pd.json_normalize(response_data['outbreaks'])
    outbreak_ids = outbreak_df['outbreakId'].tolist()

    quantitativeData = pd.json_normalize(response_data['quantitativeData'])
    measures = pd.json_normalize(response_data['measures'])

    common_df = create_common_info(response_data)
    measures_df = extract_measures_info(measures)
    quantitaive_df = create_quantitative_dataframe(quantitativeData)
    summary = pd.concat([common_df, measures_df, quantitaive_df], axis=1)

    processor1 = Qualitative_DataFormat(information_data, Country, ref_num, diagnostic_data, diag_data, control_data, ReportScraper.is_first)
    Integrated_Info = processor1.integrate_information()
    time.sleep(5)

    processor2 = SummaryEpiDataFormat(epi_data, cont_data, self.ws, ReportScraper.is_first)
    Summary_content = processor2.get_summary_content()
    time.sleep(5)

    processor3 = QuantitativeDataFormat(quantitative_data, self.ws, ReportScraper.is_first)
    Quant_data = processor3.get_summary_content()
    time.sleep(5)

    Data = []
    if ReportScraper.is_first:
      for f in range(len(Integrated_Info)):
        Data += [Integrated_Info[f] + Summary_content[f] + Quant_data[f]]
    else:
      Data = [Integrated_Info + Summary_content + Quant_data]
    # セルに入力
    if is_update:
      self.ws.insert_row(Data, insert_col+1)
      inser_col += 1
    else:
      self.ss.values_append("シート1", {"valueInputOption": "USER_ENTERED"}, {"values": Data})
    if ReportScraper.is_first:
      col = self.ws.findall('Quantitative')
      col_last = col[-1].col
      self.ws.add_cols(col_last)
      ReportScraper.is_first = False


    # 次のリストへ
    number += 1


for data in extracted_previewdata:
    print(data)

### 確認cURL

In [ ]:
%%shell
curl 'https://wahis.woah.org/api/v1/pi/review/event/5593/all-information?language=en' \
  -H 'accept: application/json' \
  -H 'content-type: application/json'


{"event":{"eventId":5593,"isForCountry":false,"isAquatic":false,"country":{"areaId":253,"name":"Zimbabwe","isoCode":"ZWE"},"disease":{"diseaseId":437,"name":"Foot and mouth disease virus (Inf. with) ","group":"Multiple species","category":"Listed disease","isDomestic":false,"isAquatic":false,"selfDeclaration":false},"causalAgent":{"agentId":131,"name":"Foot and mouth disease virus","type":"Virus"},"subType":{"disease":null,"diseaseType":{"id":115,"keyValue":"pending","translation":"Pending","description":null}},"clinicalSigns":true,"startedOn":"2024-03-08T00:00:00.000+00:00","confirmOn":"2024-03-11T00:00:00.000+00:00","lastOccurrence":"2023-01-12T00:00:00.000+00:00","endedOn":null,"reason":{"id":280,"keyValue":"reccurence disease","translation":"Recurrence of an eradicated disease","description":null},"eventStatus":{"id":136,"keyValue":"ongoing","translation":"On-going","description":null},"eventComment":{"original":"There is an outbreak that is affecting neighbouring districts. The ca

In [ ]:
%%shell
curl 'https://wahis.woah.org/api/v1/pi/review/event/5593/outbreak/136989/all-information?language=en' \
  -H 'accept: application/json' \
  -H 'content-type: application/json'

{"outbreak":{"outbreakId":136989,"areaId":28552,"oieReference":"ob_136989","nationalReference":null,"disease":null,"latitude":-19.44568,"longitude":32.52183,"location":"Rimiti","isLocationApprox":false,"epiUnitType":{"id":141,"keyValue":"Village","translation":"Village","description":null},"isCluster":false,"clusterCount":1,"startDate":"2024-06-07T00:00:00.000+00:00","endDate":null,"createdByReportId":168334,"lastUpdateReportId":168334,"description":{"original":"Cattle of varying age groups sharing the same grazing and watering points.","translation":"Cattle of varying age groups sharing the same grazing and watering points."}},"adminDivisions":[{"areaId":4025,"name":"Manicaland","adminLevel":1,"parentAreaId":253},{"areaId":28552,"name":"Mutare","adminLevel":2,"parentAreaId":4025}],"quantityUnit":{"id":56,"keyValue":"Animal","translation":"Animal","description":null},"speciesQuantities":[{"createdByCurrentReport":true,"totalQuantities":{"speciesId":1055,"speciesName":"Cattle","isWild":

In [ ]:
%%shell
#curl -i 'https://example.com/api/data?disease=Foot%20and%20mouth%20disease%20virus%20(Inf.%20with)'
curl 'https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en' \
  -H 'accept: application/json' \
  -H 'accept-language: en' \
  -H 'access-control-allow-headers: X-Requested-With, Content-Type, Origin, Authorization, Accept,Client-Security-Token, Accept-Encoding, accept-language, type, authorizationToken, methodArn' \
  -H 'access-control-allow-methods: POST, GET, OPTIONS, DELETE, PUT' \
  -H 'access-control-allow-origin: *' \
  -H 'authorizationtoken;' \
  -H 'cache-control: no-cache' \
  -H 'clientid: OIEwebsite' \
  -H 'content-type: application/json' \
  -H 'cookie: _ga=GA1.1.150018625.1719561495; cf_clearance=xpzDToaG4UFk1PVyjzgFd9a81hGXdTCalSWN2tpgpPA-1720721829-1.0.1.1-i9Ihldz1pXwEvU2vhjoJ_NJwvQR.q.zbB.nafBPy19Jzlh9GUgGQP7gJ6CwCkZS76DHWxUruE61175A8TuJDkA; _ga_X4NWRQJH7P=GS1.1.1720720245.3.1.1720721842.60.0.0' \
  -H 'env: PRD' \
  -H 'expires: Sat, 01 Jan 2000 00:00:00 GMT' \
  -H 'origin: https://wahis.woah.org' \
  -H 'pragma: no-cache' \
  -H 'priority: u=1, i' \
  -H 'referer: https://wahis.woah.org/' \
  -H 'sec-ch-ua: "Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "macOS"' \
  -H 'sec-fetch-dest: empty' \
  -H 'sec-fetch-mode: cors' \
  -H 'sec-fetch-site: same-origin' \
  -H 'security-token;' \
  -H 'token: #PIPRD202006#' \
  -H 'type: REQUEST' \
  -H 'user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36' \
  -H 'userid;' \
  -H 'x-content-type-options: nosniff' \
  -H 'x-frame-options: sameorigin' \
  --data-raw '{"eventIds":[],"reportIds":[],"countries":[],"firstDiseases":[437],"secondDiseases":[],"typeStatuses":[],"reasons":[],"eventStatuses":[],"reportTypes":[],"reportStatuses":[],"eventStartDate":null,"submissionDate":null,"animalTypes":[],"sortColumn":"submissionDate","sortOrder":"desc","pageSize":100,"pageNumber":0}'

{"list":[{"reportId":168176,"eventId":5658,"country":"South Africa","disease":"Foot and mouth disease virus (Inf. with) ","subType":"SAT 3","eventStartDate":"2024-04-14T00:00:00.000+00:00","eventStatus":"On-going","reason":"First occurrence in a zone or a compartment","reportType":"FUR","reportStatus":"Validated","submissionDate":"2024-07-10T12:12:54.575+00:00","reportNumber":7,"isAquatic":false},{"reportId":168003,"eventId":5735,"country":"South Africa","disease":"Foot and mouth disease virus (Inf. with) ","subType":"SAT 2","eventStartDate":"2024-04-30T00:00:00.000+00:00","eventStatus":"On-going","reason":"New strain in a zone or a compartment","reportType":"IN","reportStatus":"Validated","submissionDate":"2024-07-05T10:37:56.349+00:00","reportNumber":0,"isAquatic":false},{"reportId":167158,"eventId":5487,"country":"South Africa","disease":"Foot and mouth disease virus (Inf. with) ","subType":"SAT 1","eventStartDate":"2024-01-04T00:00:00.000+00:00","eventStatus":"Resolved","reason":"R

In [ ]:
%%shell
curl 'https://wahis.woah.org/api/v1/pi/review/event/5658/all-information?language=en' \
  -H 'accept: application/json' \
  -H 'accept-language: en' \
  -H 'access-control-allow-headers: X-Requested-With, Content-Type, Origin, Authorization, Accept,Client-Security-Token, Accept-Encoding, accept-language, type, authorizationToken, methodArn' \
  -H 'access-control-allow-methods: POST, GET, OPTIONS, DELETE, PUT' \
  -H 'access-control-allow-origin: *' \
  -H 'authorizationtoken;' \
  -H 'cache-control: no-cache' \
  -H 'clientid: OIEwebsite' \
  -H 'content-type: application/json' \
  -H 'cookie: _ga=GA1.1.150018625.1719561495; cf_clearance=xpzDToaG4UFk1PVyjzgFd9a81hGXdTCalSWN2tpgpPA-1720721829-1.0.1.1-i9Ihldz1pXwEvU2vhjoJ_NJwvQR.q.zbB.nafBPy19Jzlh9GUgGQP7gJ6CwCkZS76DHWxUruE61175A8TuJDkA; _ga_X4NWRQJH7P=GS1.1.1720775534.7.1.1720775562.32.0.0' \
  -H 'env: PRD' \
  -H 'expires: Sat, 01 Jan 2000 00:00:00 GMT' \
  -H 'pragma: no-cache' \
  -H 'priority: u=1, i' \
  -H 'referer: https://wahis.woah.org/' \
  -H 'sec-ch-ua: "Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "macOS"' \
  -H 'sec-fetch-dest: empty' \
  -H 'sec-fetch-mode: cors' \
  -H 'sec-fetch-site: same-origin' \
  -H 'security-token;' \
  -H 'token: #PIPRD202006#' \
  -H 'type: REQUEST' \
  -H 'user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36' \
  -H 'userid;' \
  -H 'x-content-type-options: nosniff' \
  -H 'x-frame-options: sameorigin'

{"event":{"eventId":5658,"isForCountry":false,"isAquatic":false,"country":{"areaId":251,"name":"South Africa","isoCode":"ZAF"},"disease":{"diseaseId":437,"name":"Foot and mouth disease virus (Inf. with) ","group":"Multiple species","category":"Listed disease","isDomestic":false,"isAquatic":false,"selfDeclaration":false},"causalAgent":{"agentId":131,"name":"Foot and mouth disease virus","type":"Virus"},"subType":{"disease":{"diseaseId":838,"name":"SAT 3","group":null,"category":"Listed disease","isDomestic":false,"isAquatic":false,"selfDeclaration":false},"diseaseType":null},"clinicalSigns":true,"startedOn":"2024-04-14T00:00:00.000+00:00","confirmOn":"2024-05-02T00:00:00.000+00:00","lastOccurrence":null,"endedOn":null,"reason":{"id":279,"keyValue":"First occ in zone","translation":"First occurrence in a zone or a compartment","description":null},"eventStatus":{"id":136,"keyValue":"ongoing","translation":"On-going","description":null},"eventComment":{"original":"Note on 05/07/2024: Outbr

In [ ]:
%%shell
curl 'https://wahis.woah.org/api/v1/pi/review/report/168002/event-information?language=en' \
  -H 'accept: application/json' \
  -H 'accept-language: en' \
  -H 'access-control-allow-headers: X-Requested-With, Content-Type, Origin, Authorization, Accept,Client-Security-Token, Accept-Encoding, accept-language, type, authorizationToken, methodArn' \
  -H 'access-control-allow-methods: POST, GET, OPTIONS, DELETE, PUT' \
  -H 'access-control-allow-origin: *' \
  -H 'authorizationtoken;' \
  -H 'cache-control: no-cache' \
  -H 'clientid: OIEwebsite' \
  -H 'content-type: application/json' \
  -H 'cookie: _ga=GA1.1.150018625.1719561495; cf_clearance=xpzDToaG4UFk1PVyjzgFd9a81hGXdTCalSWN2tpgpPA-1720721829-1.0.1.1-i9Ihldz1pXwEvU2vhjoJ_NJwvQR.q.zbB.nafBPy19Jzlh9GUgGQP7gJ6CwCkZS76DHWxUruE61175A8TuJDkA; _ga_X4NWRQJH7P=GS1.1.1720720245.3.1.1720721842.60.0.0' \
  -H 'env: PRD' \
  -H 'expires: Sat, 01 Jan 2000 00:00:00 GMT' \
  -H 'pragma: no-cache' \
  -H 'priority: u=1, i' \
  -H 'referer: https://wahis.woah.org/' \
  -H 'sec-ch-ua: "Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "macOS"' \
  -H 'sec-fetch-dest: empty' \
  -H 'sec-fetch-mode: cors' \
  -H 'sec-fetch-site: same-origin' \
  -H 'security-token;' \
  -H 'token: #PIPRD202006#' \
  -H 'type: REQUEST' \
  -H 'user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36' \
  -H 'userid;' \
  -H 'x-content-type-options: nosniff' \
  -H 'x-frame-options: sameorigin'

{"eventId":5658,"isForCountry":false,"isAquatic":false,"country":{"areaId":251,"name":"South Africa","isoCode":"ZAF"},"disease":{"diseaseId":437,"name":"Foot and mouth disease virus (Inf. with) ","group":"Multiple species","category":"Listed disease","isDomestic":false,"isAquatic":false,"selfDeclaration":false},"causalAgent":{"agentId":131,"name":"Foot and mouth disease virus","type":"Virus"},"subType":{"disease":{"diseaseId":838,"name":"SAT 3","group":null,"category":"Listed disease","isDomestic":false,"isAquatic":false,"selfDeclaration":false},"diseaseType":null},"clinicalSigns":true,"startedOn":"2024-04-14T00:00:00.000+00:00","confirmOn":"2024-05-02T00:00:00.000+00:00","lastOccurrence":null,"endedOn":null,"reason":{"id":279,"keyValue":"First occ in zone","translation":"First occurrence in a zone or a compartment","description":null},"eventStatus":{"id":136,"keyValue":"ongoing","translation":"On-going","description":null},"eventComment":{"original":"Note on 05/07/2024: Outbreak OB_13

In [ ]:
extracted_previewdata = []
number = 0
response = send_request_with_cookie(PQ_param, 'GET', [extracted_listdata[number]['eventId']])
response_data = response.json()
flat_data = pd.json_normalize(response_data)

In [ ]:
browser = HeadlessBrowser()
FQ_param, PQ_param, DQ_param = browser.visit_url("Foot and mouth disease")

KeyboardInterrupt: 

In [ ]:
print(FQ_param)
print(PQ_param)
print(DQ_param)

In [ ]:
url = 'https://wahis.woah.org/api/v1/pi/country/list?language=en' # 国ID
url = 'https://wahis.woah.org/api/v1/pi/disease/first-level-filters?language=en' # 疾病ID
url = 'https://wahis.woah.org/api/v1/pi/disease/second-level-filters?language=en' # サブタイプID
url = 'https://wahis.woah.org/api/v1/pi/catalog/report-reason/list?language=en' # reason ID
url = 'https://wahis.woah.org/api/v1/pi/catalog/event-status/list?language=en' # event status ID
url = 'https://wahis.woah.org/api/v1/pi/catalog/report-status/list?language=en' # Draft ID
url = 'https://wahis.woah.org/api/v1/pi/country/list-geo-region?language=en' # region ID
url = 'https://wahis.woah.org/api/v1/pi/event/reports-count' # Report ID



response = requests.get(url)

# レスポンスの確認
print(response.json())

# 余裕があったらやりたい
- scikit-learnでのWebappづくり（？）（収集条件及び出力形式（拡張子・summary/detail）が指定可能、あたり？）
- Webapp化に伴いエラー発生時のdriver再起動などの調整